In [ ]:
import os
import json
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import logging
import cv2
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram import Update
import asyncio
import nest_asyncio
from google.colab import drive

# --- Khởi tạo môi trường ---
nest_asyncio.apply()
drive.mount('/content/drive', force_remount=True)

# Cấu hình logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Token Telegram (THAY BẰNG TOKEN THẬT)
TELEGRAM_TOKEN = "7604789951:AAGfAXF6mwzrajv8r_YDvIoQKiFc6rKHJ64"

# Cho phép torch serialization
torch.serialization.add_safe_globals([np.dtype, np._core.multiarray.scalar])

# --- Cấu hình ánh xạ cột ---
column_mapping = {
    'blood_pressure': 'd1_mbp_max', 'glucose_level': 'd1_glucose_max', 'heart_rate': 'd1_heartrate_max',
    'blood_sugar': 'd1_glucose_min', 'age': 'age', 'bmi': 'bmi', 'gender': 'gender',
    'cholesterol': 'cholesterol_total', 'triglycerides': 'triglycerides_level',
    'ldl': None, 'hdl': None
}

# --- Xử lý ảnh ---
def crop_image(img, tol=7):
    """Cắt ảnh dựa trên ngưỡng xám để loại bỏ viền trống."""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    mask = gray > tol
    if mask.sum() == 0:
        return img
    rows = mask.any(axis=1)
    cols = mask.any(axis=0)
    return img[np.ix_(rows, cols)]

def preprocess_image(image, sigmaX=10, img_size=224):
    """Tiền xử lý ảnh: cắt, resize, tăng cường độ tương phản."""
    if isinstance(image, Image.Image):
        image = np.array(image.convert('RGB'))
    image = crop_image(image, tol=7)
    image = cv2.resize(image, (img_size, img_size))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)
    return image

# --- Lớp TensorFlow tùy chỉnh ---
class CustomGridDropout(tf.keras.layers.Layer):
    def __init__(self, ratio=0.3, holes_number=4, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio
        self.holes_number = holes_number
        self.p = p

    def call(self, inputs, training=None):
        if not training:
            return inputs
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        batch_size = tf.shape(inputs)[0]
        feature_dim = tf.shape(inputs)[1]
        hole_size = tf.maximum(1, tf.cast(tf.cast(feature_dim, tf.float32) * self.ratio, tf.int32))
        mask = tf.ones_like(inputs, dtype=tf.float32)
        random_probs = tf.random.uniform([self.holes_number], 0, 1)
        active_holes = tf.cast(random_probs < self.p, tf.int32)
        hole_indices = tf.range(self.holes_number)
        start_indices = (hole_indices * feature_dim) // self.holes_number
        end_indices = tf.minimum(start_indices + hole_size, feature_dim)
        all_indices = []
        for i in range(self.holes_number):
            if active_holes[i]:
                indices = tf.stack([
                    tf.tile(tf.range(batch_size), [end_indices[i] - start_indices[i]]),
                    tf.repeat(tf.range(start_indices[i], end_indices[i]), batch_size)
                ], axis=1)
                all_indices.append(indices)
        if all_indices:
            all_indices = tf.concat(all_indices, axis=0)
            updates = tf.zeros([tf.shape(all_indices)[0]], dtype=tf.float32)
            mask = tf.tensor_scatter_nd_update(mask, all_indices, updates)
        return inputs * mask

    def get_config(self):
        config = super().get_config()
        config.update({"ratio": self.ratio, "holes_number": self.holes_number, "p": self.p})
        return config

class MemoryAugmentedLayer(tf.keras.layers.Layer):
    def __init__(self, memory_size, memory_dim, **kwargs):
        super().__init__(**kwargs)
        self.memory_size = memory_size
        self.memory_dim = memory_dim

    def build(self, input_shape):
        self.memory = self.add_weight(
            shape=(self.memory_size, self.memory_dim), initializer='zeros', trainable=False, dtype=tf.float32
        )
        super().build(input_shape)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        memory_size = tf.shape(self.memory)[0]
        memory_sliced = tf.cond(
            batch_size > memory_size,
            lambda: tf.tile(self.memory, [(batch_size + memory_size - 1) // memory_size, 1])[:batch_size],
            lambda: self.memory[:batch_size]
        )
        return tf.reduce_mean(tf.stack([inputs, memory_sliced], axis=0), axis=0)

    def get_config(self):
        config = super().get_config()
        config.update({'memory_size': self.memory_size, 'memory_dim': self.memory_dim})
        return config

class GradientReversalLayer(Layer):
    def __init__(self, lambda_=1.0, **kwargs):
        super().__init__(**kwargs)
        self.lambda_ = lambda_

    def call(self, inputs, training=None):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        return inputs if not training else tf.math.multiply(-self.lambda_, inputs)

    def get_config(self):
        config = super().get_config()
        config.update({"lambda_": self.lambda_})
        return config

# --- Mô hình PyTorch ---
class AdvancedMLP(nn.Module):
    """Mô hình MLP để dự đoán mức độ nghiêm trọng."""
    def __init__(self, input_size=20, hidden_sizes=[243, 128], num_classes=5, dropout_rate=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.LeakyReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# --- Xử lý dữ liệu lâm sàng ---
def load_data(file_path, max_rows=10000):
    """Tải dữ liệu từ CSV, tối ưu kiểu dữ liệu."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File không tồn tại: {file_path}")
    usecols = [
        'age', 'bmi', 'gender', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_resprate_max', 'd1_resprate_min',
        'd1_spo2_max', 'd1_spo2_min', 'd1_temp_max', 'd1_temp_min', 'd1_mbp_max', 'd1_mbp_min',
        'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min',
        'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_bilirubin_max', 'd1_bilirubin_min',
        'd1_lactate_max', 'd1_lactate_min', 'diabetes_mellitus', 'cirrhosis', 'hepatic_failure',
        'aids', 'immunosuppression', 'gcs_total', 'pao2_fio2_ratio', 'cholesterol_total', 'triglycerides_level'
    ]
    usecols = [col for col in usecols if col in pd.read_csv(file_path, nrows=1).columns]
    df = pd.read_csv(file_path, usecols=usecols, nrows=max_rows)
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    logger.info(f"Đã tải {len(df)} dòng, {len(df.columns)} cột")
    return df

def validate_clinical_data(data):
    """Kiểm tra dữ liệu lâm sàng đầu vào."""
    errors = []
    if not isinstance(data, dict):
        return ["Dữ liệu phải là JSON hợp lệ"]
    for key, value in data.items():
        if key in ['age', 'blood_pressure', 'glucose_level', 'heart_rate', 'bmi', 'cholesterol', 'triglycerides']:
            if not isinstance(value, (int, float)) or value < 0:
                errors.append(f"Giá trị {key} phải là số không âm")
        elif key == 'gender' and value not in ['Male', 'Female']:
            errors.append("Giới tính phải là 'Male' hoặc 'Female'")
    return errors

def impute_missing_values(clinical_df, reference_df):
    """Điền giá trị thiếu bằng RandomForest."""
    df_impute = clinical_df.copy().rename(columns={k: v for k, v in column_mapping.items() if v})
    numeric_cols = reference_df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = reference_df.select_dtypes(include=['object']).columns

    for col in numeric_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c != col and c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột số: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, y_train)
            df_impute.loc[df_impute[col].isna(), col] = rf.predict(X_test_processed)
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")

    for col in categorical_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột phân loại: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, pd.Categorical(y_train).codes)
            predicted_codes = rf.predict(X_test_processed).astype(int)
            valid_categories = pd.Categorical(reference_df[col].dropna().unique())
            df_impute.loc[df_impute[col].isna(), col] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")

    return df_impute

def predict_new_columns(training_df, predict_df):
    """Dự đoán các cột mới bằng RandomForest."""
    new_columns = [col for col in training_df.columns if col not in predict_df.columns]
    logger.info(f"Dự đoán các cột: {new_columns}")
    result_df = predict_df.copy()

    for column in new_columns:
        logger.info(f"Dự đoán cột: {column}")
        X_train = training_df.drop(columns=[column])
        y_train = training_df[column].values
        X_test = result_df.copy()
        for col in training_df.columns:
            if col not in X_test.columns and col != column:
                X_test[col] = np.nan
        X_test = X_test[training_df.drop(columns=[column]).columns]

        numeric_cols = X_train.select_dtypes(include=['int32', 'float32']).columns
        categorical_cols = X_train.select_dtypes(include=['object']).columns
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
                ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
            ])
        try:
            X_train_processed = preprocessor.fit_transform(X_train).astype('float32')
            X_test_processed = preprocessor.transform(X_test).astype('float32')
            if column in numeric_cols:
                model = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, y_train)
                result_df[column] = model.predict(X_test_processed)
            else:
                model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, pd.Categorical(y_train).codes)
                predicted_codes = model.predict(X_test_processed).astype(int)
                valid_categories = pd.Categorical(training_df[column].dropna().unique())
                result_df[column] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi dự đoán cột {column}: {e}")
            result_df[column] = np.nan
    return result_df

def preprocess_clinical_data(df):
    """Tiền xử lý dữ liệu lâm sàng: điền giá trị thiếu, chuẩn hóa, mã hóa."""
    numeric_cols = df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
            ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
        ])
    X_processed = preprocessor.fit_transform(df).astype('float32')
    logger.info(f"Đặc trưng sau tiền xử lý: {X_processed.shape[1]}")
    return torch.FloatTensor(X_processed), preprocessor

def prepare_input_data(clinical_df, image_features=None, preprocessor=None, expected_input_size=20):
    """Chuẩn bị dữ liệu đầu vào cho mô hình."""
    X_processed = preprocessor.transform(clinical_df).astype('float32')
    X_processed = torch.FloatTensor(X_processed)
    logger.info(f"Đặc trưng lâm sàng sau tiền xử lý: {X_processed.shape[1]}")
    if image_features is not None and image_features.shape[0] == X_processed.shape[0]:
        X_processed = torch.cat((X_processed, torch.FloatTensor(image_features.astype('float32'))), dim=1)
        logger.info(f"Kích thước đặc trưng ảnh: {image_features.shape[1]}")
    if X_processed.shape[1] != expected_input_size:
        if X_processed.shape[1] > expected_input_size:
            X_processed = X_processed[:, :expected_input_size]
            logger.info(f"Cắt bớt còn {expected_input_size} chiều")
        else:
            padding = torch.zeros(X_processed.shape[0], expected_input_size - X_processed.shape[1])
            X_processed = torch.cat((X_processed, padding), dim=1)
            logger.info(f"Thêm padding cho {expected_input_size - X_processed.shape[1]} chiều")
    logger.info(f"Kích thước đầu vào cuối cùng: {X_processed.shape}")
    return X_processed

# --- Tải và dự đoán mô hình ---
def load_meta_model(model_path='/content/drive/MyDrive/main_clinical_model.pth'):
    """Tải mô hình lâm sàng từ file .pth."""
    try:
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
        model = AdvancedMLP(input_size=20, hidden_sizes=[243, 128], num_classes=checkpoint['num_classes'], dropout_rate=checkpoint['dropout_rate'])
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        model.eval()
        logger.info(f"Đã tải mô hình lâm sàng từ {model_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình lâm sàng: {e}")
        raise

def load_image_model(model_path='/content/drive/MyDrive/working/meta_bestqwk'):
    """Tải mô hình xử lý ảnh từ file .h5 và .json."""
    try:
        weights_path = os.path.join(model_path, "model.weights.h5")
        config_path = os.path.join(model_path, "config.json")
        with open(config_path, 'r') as f:
            model_config = json.load(f)
        model = model_from_json(json.dumps(model_config), custom_objects={
            'CustomGridDropout': CustomGridDropout, 'MemoryAugmentedLayer': MemoryAugmentedLayer, 'GradientReversalLayer': GradientReversalLayer
        })
        model.load_weights(weights_path)
        model.trainable = False
        logger.info(f"Đã tải mô hình ảnh từ {weights_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình ảnh: {e}")
        raise

def extract_features_from_image(image_path, image_model, target_dim=50):
    """Trích xuất đặc trưng từ ảnh bằng EfficientNetB1."""
    if not os.path.exists(image_path):
        logger.error(f"Ảnh không tồn tại tại {image_path}")
        return None
    try:
        image = Image.open(image_path).convert('RGB')
        processed_image = preprocess_image(image, sigmaX=10, img_size=224)
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image_tensor = transform(processed_image).unsqueeze(0)
        image_tf = tf.transpose(tf.convert_to_tensor(image_tensor.numpy(), dtype=tf.float32), perm=[0, 2, 3, 1])
        image_tf_preprocessed = efficientnet_preprocess(image_tf)
        base_model = EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
        base_model.trainable = False
        raw_features = base_model.predict(image_tf_preprocessed, batch_size=1, verbose=0)
        reduce_to_target_dim = Sequential([tf.keras.layers.Input(shape=(raw_features.shape[-1],)), tf.keras.layers.Dense(target_dim, use_bias=False, dtype=tf.float32)])
        reduced_features = reduce_to_target_dim.predict(raw_features, batch_size=1, verbose=0)
        logger.info(f"Đã trích xuất đặc trưng ảnh: shape {reduced_features.shape}")
        return reduced_features.astype('float32')
    except Exception as e:
        logger.error(f"Lỗi trích xuất đặc trưng ảnh: {e}")
        return None

def predict_severity(clinical_df, image_path=None, clinical_model_path='/content/drive/MyDrive/main_clinical_model.pth',
                     image_model_path='/content/drive/MyDrive/working/meta_bestqwk', preprocessor=None):
    """Dự đoán mức độ nghiêm trọng từ dữ liệu lâm sàng và ảnh."""
    try:
        clinical_model = load_meta_model(clinical_model_path)
        image_model = load_image_model(image_model_path) if image_path else None
        image_features = extract_features_from_image(image_path, image_model, target_dim=50) if image_path and image_model else None
        X_input = prepare_input_data(clinical_df, image_features, preprocessor, expected_input_size=20)
        logger.info(f"Dự đoán với kích thước đầu vào: {X_input.shape}")
        with torch.no_grad():
            outputs = clinical_model(X_input)
            _, predicted = torch.max(outputs, 1)
        return predicted.numpy()
    except Exception as e:
        logger.error(f"Lỗi dự đoán: {e}")
        raise

# --- Xử lý Telegram ---
async def start(update, context):
    """Hiển thị hướng dẫn sử dụng bot."""
    logger.info("Nhận lệnh /start")
    await update.message.reply_text(
        "👋 *Chào bạn!* Tôi là bot dự đoán mức độ nghiêm trọng bệnh dựa trên dữ liệu lâm sàng và hình ảnh.\n\n"
        "📋 *Cách sử dụng:*\n"
        "1️⃣ Gửi dữ liệu lâm sàng dạng JSON (như tuổi, huyết áp, cholesterol...).\n"
        "2️⃣ Gửi một hình ảnh (nếu có).\n"
        "3️⃣ Nhận kết quả: mức độ nghiêm trọng (0-4) và dữ liệu bổ sung.\n\n"
        "📌 *Ví dụ JSON:*\n"
        "```json\n"
        "{\n"
        "  \"age\": 45,\n"
        "  \"blood_pressure\": 130,\n"
        "  \"glucose_level\": 90,\n"
        "  \"heart_rate\": 75,\n"
        "  \"bmi\": 24,\n"
        "  \"gender\": \"Male\",\n"
        "  \"cholesterol\": 200,\n"
        "  \"triglycerides\": 150\n"
        "}\n"
        "```\n"
        "💡 Gửi `/skip_photo` nếu muốn bỏ qua ảnh.\n"
        "Hãy gửi JSON để bắt đầu!"
    )

async def skip_photo(update, context):
    """Cho phép bỏ qua gửi ảnh và dự đoán chỉ với dữ liệu lâm sàng."""
    logger.info("Nhận lệnh /skip_photo")
    if 'clinical_data' not in context.user_data:
        await update.message.reply_text("⚠️ Vui lòng gửi dữ liệu lâm sàng trước!")
        return
    await update.message.reply_text("📸 Đã bỏ qua ảnh. Đang xử lý dữ liệu...")
    await process_data(update, context, image_path=None)

async def handle_message(update, context):
    """Xử lý tin nhắn JSON từ người dùng."""
    logger.info("Nhận tin nhắn văn bản")
    try:
        clinical_data = json.loads(update.message.text)
        errors = validate_clinical_data(clinical_data)
        if errors:
            await update.message.reply_text(f"⚠️ Lỗi dữ liệu:\n" + "\n".join(errors))
            return
        if not any(key in clinical_data for key in column_mapping):
            await update.message.reply_text("⚠️ JSON phải chứa ít nhất một chỉ số như age, blood_pressure, v.v.")
            return
        context.user_data['clinical_data'] = clinical_data
        await update.message.reply_text(
            "✅ Đã nhận dữ liệu lâm sàng.\n"
            "📸 Vui lòng gửi hình ảnh hoặc gửi `/skip_photo` để bỏ qua."
        )
        logger.info(f"Đã nhận dữ liệu lâm sàng: {clinical_data}")
    except json.JSONDecodeError:
        await update.message.reply_text("❌ Dữ liệu không đúng định dạng JSON. Vui lòng gửi lại.")
    except Exception as e:
        await update.message.reply_text(f"❌ Lỗi: {str(e)}")
        logger.error(f"Lỗi xử lý tin nhắn: {e}")

async def handle_photo(update, context):
    """Xử lý ảnh từ người dùng."""
    logger.info("Nhận ảnh")
    if 'clinical_data' not in context.user_data:
        await update.message.reply_text("⚠️ Vui lòng gửi dữ liệu lâm sàng trước!")
        return

    photo = update.message.photo[-1]
    file = await context.bot.get_file(photo.file_id)
    image_path = f"/tmp/{photo.file_id}.jpg"
    await file.download_to_drive(image_path)

    await update.message.reply_text("📸 Đã nhận ảnh. Đang xử lý...")
    await process_data(update, context, image_path)

async def process_data(update, context, image_path):
    """Xử lý dữ liệu lâm sàng và ảnh, trả kết quả."""
    clinical_data = context.user_data['clinical_data']
    clinical_df = pd.DataFrame([clinical_data])
    training_file_path = '/content/drive/MyDrive/TrainingWiDS2021_filled.csv'
    clinical_model_path = '/content/drive/MyDrive/main_clinical_model.pth'
    image_model_path = '/content/drive/MyDrive/working/meta_bestqwk'

    try:
        training_df = load_data(training_file_path, max_rows=10000)
        imputed_df = impute_missing_values(clinical_df, training_df)
        extended_df = predict_new_columns(training_df, imputed_df)
        X_processed, preprocessor = preprocess_clinical_data(extended_df)
        severity = predict_severity(extended_df, image_path, clinical_model_path, image_model_path, preprocessor)

        # Thêm mức độ nghiêm trọng vào dữ liệu
        extended_df['predicted_severity'] = severity[0]

        # Chuẩn bị phản hồi
        severity_map = {0: "Rất nhẹ", 1: "Nhẹ", 2: "Trung bình", 3: "Nặng", 4: "Rất nặng"}
        severity_label = severity_map.get(severity[0], "Không xác định")
        await update.message.reply_text(
            f"🎯 *Kết quả dự đoán*\n"
            f"🔹 Mức độ nghiêm trọng: *{severity[0]}* ({severity_label})\n"
            f"🔹 Dựa trên dữ liệu lâm sàng{' và ảnh' if image_path else ''}."
        )

        # Hiển thị dữ liệu bổ sung dạng bảng
        table = "*Dữ liệu bổ sung*\n"
        table += "| Chỉ số | Giá trị |\n"
        table += "|---|---|\n"
        for col, val in extended_df.iloc[0].items():
            if pd.isna(val):
                continue
            val_str = f"{val:.2f}" if isinstance(val, (int, float)) else str(val)
            table += f"| {col} | {val_str} |\n"
        await update.message.reply_text(table, parse_mode='Markdown')

        # Lưu vào CSV
        extended_df.to_csv('/content/drive/MyDrive/extended_clinical_data.csv', index=False)
        await update.message.reply_text(
            "💾 Dữ liệu đã được lưu vào: `/content/drive/MyDrive/extended_clinical_data.csv`\n"
            "📋 Gửi JSON mới để tiếp tục!"
        )
    except Exception as e:
        await update.message.reply_text(f"❌ Dự đoán thất bại: {str(e)}")
        logger.error(f"Lỗi dự đoán: {e}")
    finally:
        context.user_data.clear()
        if image_path and os.path.exists(image_path):
            os.remove(image_path)

async def error_handler(update, context):
    """Xử lý lỗi Telegram."""
    logger.warning(f'Update "{update}" gây lỗi "{context.error}"')

def main():
    """Chạy bot Telegram."""
    logger.info("Khởi động bot")
    os.environ['TZ'] = 'UTC'
    try:
        import time
        time.tzset()
        logger.info("Đặt múi giờ UTC")
    except AttributeError:
        logger.warning("Không thể đặt múi giờ")

    application = Application.builder().token(TELEGRAM_TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("skip_photo", skip_photo))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))
    application.add_error_handler(error_handler)
    application.run_polling(allowed_updates=Update.ALL_TYPES)

if __name__ == '__main__':
    main() chỉnh lại mã trên dựa mã dưới

IndentationError: expected an indented block after 'if' statement on line 2131 (<ipython-input-4-657af0013c5d>, line 2132)

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import logging
import cv2
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram import Update
import asyncio
import nest_asyncio
from google.colab import drive

# --- Khởi tạo môi trường ---
nest_asyncio.apply()
drive.mount('/content/drive', force_remount=True)

# Cấu hình logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Token Telegram
TELEGRAM_TOKEN = "7604789951:AAGfAXF6mwzrajv8r_YDvIoQKiFc6rKHJ64"

# Cho phép torch serialization
torch.serialization.add_safe_globals([np.dtype, np._core.multiarray.scalar])

# --- Cấu hình ánh xạ cột ---
column_mapping = {
    'blood_pressure': 'd1_mbp_max', 'glucose_level': 'd1_glucose_max', 'heart_rate': 'd1_heartrate_max',
    'blood_sugar': 'd1_glucose_min', 'age': 'age', 'bmi': 'bmi', 'gender': 'gender',
    'cholesterol': 'cholesterol_total', 'triglycerides': 'triglycerides_level',
    'ldl': None, 'hdl': None
}

# --- Xử lý ảnh ---
def crop_image(img, tol=7):
    """Cắt ảnh dựa trên ngưỡng xám để loại bỏ viền trống."""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    mask = gray > tol
    if mask.sum() == 0:
        return img
    rows = mask.any(axis=1)
    cols = mask.any(axis=0)
    return img[np.ix_(rows, cols)]

def preprocess_image(image, sigmaX=10, img_size=224):
    """Tiền xử lý ảnh: cắt, resize, tăng cường độ tương phản."""
    if isinstance(image, Image.Image):
        image = np.array(image.convert('RGB'))
    image = crop_image(image, tol=7)
    image = cv2.resize(image, (img_size, img_size))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)
    return image

# --- Lớp TensorFlow tùy chỉnh ---
class CustomGridDropout(tf.keras.layers.Layer):
    def __init__(self, ratio=0.3, holes_number=4, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio
        self.holes_number = holes_number
        self.p = p

    def call(self, inputs, training=None):
        if not training:
            return inputs
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        batch_size = tf.shape(inputs)[0]
        feature_dim = tf.shape(inputs)[1]
        hole_size = tf.maximum(1, tf.cast(tf.cast(feature_dim, tf.float32) * self.ratio, tf.int32))
        mask = tf.ones_like(inputs, dtype=tf.float32)
        random_probs = tf.random.uniform([self.holes_number], 0, 1)
        active_holes = tf.cast(random_probs < self.p, tf.int32)
        hole_indices = tf.range(self.holes_number)
        start_indices = (hole_indices * feature_dim) // self.holes_number
        end_indices = tf.minimum(start_indices + hole_size, feature_dim)
        all_indices = []
        for i in range(self.holes_number):
            if active_holes[i]:
                indices = tf.stack([
                    tf.tile(tf.range(batch_size), [end_indices[i] - start_indices[i]]),
                    tf.repeat(tf.range(start_indices[i], end_indices[i]), batch_size)
                ], axis=1)
                all_indices.append(indices)
        if all_indices:
            all_indices = tf.concat(all_indices, axis=0)
            updates = tf.zeros([tf.shape(all_indices)[0]], dtype=tf.float32)
            mask = tf.tensor_scatter_nd_update(mask, all_indices, updates)
        return inputs * mask

    def get_config(self):
        config = super().get_config()
        config.update({"ratio": self.ratio, "holes_number": self.holes_number, "p": self.p})
        return config

class MemoryAugmentedLayer(tf.keras.layers.Layer):
    def __init__(self, memory_size, memory_dim, **kwargs):
        super().__init__(**kwargs)
        self.memory_size = memory_size
        self.memory_dim = memory_dim

    def build(self, input_shape):
        self.memory = self.add_weight(
            shape=(self.memory_size, self.memory_dim), initializer='zeros', trainable=False, dtype=tf.float32
        )
        super().build(input_shape)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        memory_size = tf.shape(self.memory)[0]
        memory_sliced = tf.cond(
            batch_size > memory_size,
            lambda: tf.tile(self.memory, [(batch_size + memory_size - 1) // memory_size, 1])[:batch_size],
            lambda: self.memory[:batch_size]
        )
        return tf.reduce_mean(tf.stack([inputs, memory_sliced], axis=0), axis=0)

    def get_config(self):
        config = super().get_config()
        config.update({'memory_size': self.memory_size, 'memory_dim': self.memory_dim})
        return config

class GradientReversalLayer(Layer):
    def __init__(self, lambda_=1.0, **kwargs):
        super().__init__(**kwargs)
        self.lambda_ = lambda_

    def call(self, inputs, training=None):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        return inputs if not training else tf.math.multiply(-self.lambda_, inputs)

    def get_config(self):
        config = super().get_config()
        config.update({"lambda_": self.lambda_})
        return config

# --- Mô hình PyTorch ---
class AdvancedMLP(nn.Module):
    """Mô hình MLP để dự đoán mức độ nghiêm trọng."""
    def __init__(self, input_size=20, hidden_sizes=[243, 128], num_classes=5, dropout_rate=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.LeakyReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# --- Xử lý dữ liệu lâm sàng ---
def load_data(file_path, max_rows=10000):
    """Tải dữ liệu từ CSV, tối ưu kiểu dữ liệu."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File không tồn tại: {file_path}")
    usecols = [
        'age', 'bmi', 'gender', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_resprate_max', 'd1_resprate_min',
        'd1_spo2_max', 'd1_spo2_min', 'd1_temp_max', 'd1_temp_min', 'd1_mbp_max', 'd1_mbp_min',
        'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min',
        'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_bilirubin_max', 'd1_bilirubin_min',
        'd1_lactate_max', 'd1_lactate_min', 'diabetes_mellitus', 'cirrhosis', 'hepatic_failure',
        'aids', 'immunosuppression', 'gcs_total', 'pao2_fio2_ratio', 'cholesterol_total', 'triglycerides_level'
    ]
    usecols = [col for col in usecols if col in pd.read_csv(file_path, nrows=1).columns]
    df = pd.read_csv(file_path, usecols=usecols, nrows=max_rows)
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    logger.info(f"Đã tải {len(df)} dòng, {len(df.columns)} cột")
    return df

def validate_clinical_data(data):
    """Kiểm tra dữ liệu lâm sàng đầu vào."""
    errors = []
    if not isinstance(data, dict):
        return ["Dữ liệu phải là JSON hợp lệ"]
    for key, value in data.items():
        if key in ['age', 'blood_pressure', 'glucose_level', 'heart_rate', 'bmi', 'cholesterol', 'triglycerides']:
            if not isinstance(value, (int, float)) or value < 0:
                errors.append(f"Giá trị {key} phải là số không âm")
        elif key == 'gender' and value not in ['Male', 'Female']:
            errors.append("Giới tính phải là 'Male' hoặc 'Female'")
    return errors

def impute_missing_values(clinical_df, reference_df):
    """Điền giá trị thiếu bằng RandomForest."""
    df_impute = clinical_df.copy().rename(columns={k: v for k, v in column_mapping.items() if v})
    numeric_cols = reference_df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = reference_df.select_dtypes(include=['object']).columns

    for col in numeric_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c != col and c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột số: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, y_train)
            df_impute.loc[df_impute[col].isna(), col] = rf.predict(X_test_processed)
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")

    for col in categorical_cols:
        if col not in df_impute.columns or not df_impute[col].isna().any():
            continue
        known_cols = [c for c in numeric_cols if c in df_impute.columns]
        if not known_cols:
            logger.warning(f"Bỏ qua {col}: không có đặc trưng hợp lệ")
            continue
        logger.info(f"Điền giá trị thiếu cho cột phân loại: {col}")
        X_train = reference_df[known_cols].dropna()
        y_train = reference_df[col].loc[X_train.index]
        X_test = df_impute[known_cols].loc[df_impute[col].isna()]
        if len(X_test) == 0 or len(X_train) == 0:
            continue
        preprocessor = ColumnTransformer(
            transformers=[('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), known_cols)]
        )
        try:
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1)
            rf.fit(X_train_processed, pd.Categorical(y_train).codes)
            predicted_codes = rf.predict(X_test_processed).astype(int)
            valid_categories = pd.Categorical(reference_df[col].dropna().unique())
            df_impute.loc[df_impute[col].isna(), col] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi điền giá trị cho {col}: {e}")

    return df_impute

def predict_new_columns(training_df, predict_df):
    """Dự đoán các cột mới bằng RandomForest."""
    new_columns = [col for col in training_df.columns if col not in predict_df.columns]
    logger.info(f"Dự đoán các cột: {new_columns}")
    result_df = predict_df.copy()

    for column in new_columns:
        logger.info(f"Dự đoán cột: {column}")
        X_train = training_df.drop(columns=[column])
        y_train = training_df[column].values
        X_test = result_df.copy()
        for col in training_df.columns:
            if col not in X_test.columns and col != column:
                X_test[col] = np.nan
        X_test = X_test[training_df.drop(columns=[column]).columns]

        numeric_cols = X_train.select_dtypes(include=['int32', 'float32']).columns
        categorical_cols = X_train.select_dtypes(include=['object']).columns
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
                ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
            ])
        try:
            X_train_processed = preprocessor.fit_transform(X_train).astype('float32')
            X_test_processed = preprocessor.transform(X_test).astype('float32')
            if column in numeric_cols:
                model = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, y_train)
                result_df[column] = model.predict(X_test_processed)
            else:
                model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42, n_jobs=1)
                model.fit(X_train_processed, pd.Categorical(y_train).codes)
                predicted_codes = model.predict(X_test_processed).astype(int)
                valid_categories = pd.Categorical(training_df[column].dropna().unique())
                result_df[column] = [valid_categories[i % len(valid_categories)] for i in predicted_codes]
        except Exception as e:
            logger.error(f"Lỗi dự đoán cột {column}: {e}")
            result_df[column] = np.nan
    return result_df

def preprocess_clinical_data(df):
    """Tiền xử lý dữ liệu lâm sàng: điền giá trị thiếu, chuẩn hóa, mã hóa."""
    if df.empty:
        logger.info("Không có dữ liệu lâm sàng, trả về preprocessor rỗng")
        return torch.FloatTensor(np.zeros((1, 0))), None
    numeric_cols = df.select_dtypes(include=['float32', 'int32']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', RobustScaler())]), numeric_cols),
            ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), categorical_cols)
        ])
    X_processed = preprocessor.fit_transform(df).astype('float32')
    logger.info(f"Đặc trưng sau tiền xử lý: {X_processed.shape[1]}")
    return torch.FloatTensor(X_processed), preprocessor

def prepare_input_data(clinical_df=None, image_features=None, preprocessor=None, expected_input_size=20):
    """Chuẩn bị dữ liệu đầu vào cho mô hình, hỗ trợ chỉ ảnh."""
    if clinical_df is not None and not clinical_df.empty and preprocessor is not None:
        X_processed = preprocessor.transform(clinical_df).astype('float32')
        X_processed = torch.FloatTensor(X_processed)
        logger.info(f"Đặc trưng lâm sàng sau tiền xử lý: {X_processed.shape[1]}")
    else:
        X_processed = torch.FloatTensor(np.zeros((1, 0)))  # Không có dữ liệu lâm sàng
        logger.info("Không có dữ liệu lâm sàng, sử dụng đặc trưng ảnh")

    if image_features is not None and image_features.shape[0] == 1:
        image_features_tensor = torch.FloatTensor(image_features.astype('float32'))
        X_processed = torch.cat((X_processed, image_features_tensor), dim=1) if X_processed.shape[1] > 0 else image_features_tensor
        logger.info(f"Kích thước đặc trưng ảnh: {image_features.shape[1]}")

    if X_processed.shape[1] != expected_input_size:
        if X_processed.shape[1] > expected_input_size:
            X_processed = X_processed[:, :expected_input_size]
            logger.info(f"Cắt bớt còn {expected_input_size} chiều")
        else:
            padding = torch.zeros(X_processed.shape[0], expected_input_size - X_processed.shape[1])
            X_processed = torch.cat((X_processed, padding), dim=1)
            logger.info(f"Thêm padding cho {expected_input_size - X_processed.shape[1]} chiều")
    logger.info(f"Kích thước đầu vào cuối cùng: {X_processed.shape}")
    return X_processed

# --- Tải và dự đoán mô hình ---
def load_meta_model(model_path='/content/drive/MyDrive/main_clinical_model.pth'):
    """Tải mô hình lâm sàng từ file .pth."""
    try:
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
        model = AdvancedMLP(input_size=20, hidden_sizes=[243, 128], num_classes=checkpoint['num_classes'], dropout_rate=checkpoint['dropout_rate'])
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        model.eval()
        logger.info(f"Đã tải mô hình lâm sàng từ {model_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình lâm sàng: {e}")
        raise

def load_image_model(model_path='/content/drive/MyDrive/working/meta_bestqwk'):
    """Tải mô hình xử lý ảnh từ file .h5 và .json."""
    try:
        weights_path = os.path.join(model_path, "model.weights.h5")
        config_path = os.path.join(model_path, "config.json")
        with open(config_path, 'r') as f:
            model_config = json.load(f)
        model = model_from_json(json.dumps(model_config), custom_objects={
            'CustomGridDropout': CustomGridDropout, 'MemoryAugmentedLayer': MemoryAugmentedLayer, 'GradientReversalLayer': GradientReversalLayer
        })
        model.load_weights(weights_path)
        model.trainable = False
        logger.info(f"Đã tải mô hình ảnh từ {weights_path}")
        return model
    except Exception as e:
        logger.error(f"Lỗi tải mô hình ảnh: {e}")
        raise

def extract_features_from_image(image_path, image_model, target_dim=50):
    """Trích xuất đặc trưng từ ảnh bằng EfficientNetB1."""
    if not os.path.exists(image_path):
        logger.error(f"Ảnh không tồn tại tại {image_path}")
        return None
    try:
        image = Image.open(image_path).convert('RGB')
        processed_image = preprocess_image(image, sigmaX=10, img_size=224)
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image_tensor = transform(processed_image).unsqueeze(0)
        image_tf = tf.transpose(tf.convert_to_tensor(image_tensor.numpy(), dtype=tf.float32), perm=[0, 2, 3, 1])
        image_tf_preprocessed = efficientnet_preprocess(image_tf)
        base_model = EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
        base_model.trainable = False
        raw_features = base_model.predict(image_tf_preprocessed, batch_size=1, verbose=0)
        reduce_to_target_dim = Sequential([tf.keras.layers.Input(shape=(raw_features.shape[-1],)), tf.keras.layers.Dense(target_dim, use_bias=False, dtype=tf.float32)])
        reduced_features = reduce_to_target_dim.predict(raw_features, batch_size=1, verbose=0)
        logger.info(f"Đã trích xuất đặc trưng ảnh: shape {reduced_features.shape}")
        return reduced_features.astype('float32')
    except Exception as e:
        logger.error(f"Lỗi trích xuất đặc trưng ảnh: {e}")
        return None

def predict_severity(clinical_df=None, image_path=None, clinical_model_path='/content/drive/MyDrive/main_clinical_model.pth',
                     image_model_path='/content/drive/MyDrive/working/meta_bestqwk', preprocessor=None):
    """Dự đoán mức độ nghiêm trọng từ dữ liệu lâm sàng và/hoặc ảnh."""
    if clinical_df is None and image_path is None:
        raise ValueError("Cần ít nhất dữ liệu lâm sàng hoặc ảnh để dự đoán")

    try:
        clinical_model = load_meta_model(clinical_model_path)
        image_model = load_image_model(image_model_path) if image_path else None
        image_features = extract_features_from_image(image_path, image_model, target_dim=50) if image_path and image_model else None
        if image_features is None and clinical_df is None:
            raise ValueError("Không thể trích xuất đặc trưng ảnh và không có dữ liệu lâm sàng")

        X_input = prepare_input_data(clinical_df, image_features, preprocessor, expected_input_size=20)
        logger.info(f"Dự đoán với kích thước đầu vào: {X_input.shape}")
        with torch.no_grad():
            outputs = clinical_model(X_input)
            _, predicted = torch.max(outputs, 1)
        return predicted.numpy()
    except Exception as e:
        logger.error(f"Lỗi dự đoán: {e}")
        raise

# --- Xử lý Telegram ---
async def start(update: Update, context):
    """Hiển thị hướng dẫn sử dụng bot."""
    logger.info("Nhận lệnh /start")
    await update.message.reply_text(
        "👋 *Chào bạn!* Tôi là bot dự đoán mức độ nghiêm trọng bệnh dựa trên dữ liệu lâm sàng và/hoặc hình ảnh.\n\n"
        "📋 *Cách sử dụng:*\n"
        "1️⃣ Gửi dữ liệu lâm sàng dạng JSON (như tuổi, huyết áp, cholesterol...).\n"
        "2️⃣ Gửi một hình ảnh.\n"
        "3️⃣ Hoặc chỉ gửi ảnh để dự đoán chỉ dựa trên ảnh.\n"
        "4️⃣ Nhận kết quả: mức độ nghiêm trọng (0-4) và dữ liệu bổ sung (nếu có).\n\n"
        "📌 *Ví dụ JSON:*\n"
        "```json\n"
        "{\n"
        "  \"age\": 45,\n"
        "  \"blood_pressure\": 130,\n"
        "  \"glucose_level\": 90,\n"
        "  \"heart_rate\": 75,\n"
        "  \"bmi\": 24,\n"
        "  \"gender\": \"Male\",\n"
        "  \"cholesterol\": 200,\n"
        "  \"triglycerides\": 150\n"
        "}\n"
        "```\n"
        "💡 Gửi `/skip_photo` để bỏ qua ảnh hoặc `/skip_clinical` để bỏ qua dữ liệu lâm sàng.\n"
        "Hãy gửi JSON hoặc ảnh để bắt đầu!",
        parse_mode='Markdown'
    )

async def skip_photo(update: Update, context):
    """Cho phép bỏ qua gửi ảnh và dự đoán chỉ với dữ liệu lâm sàng."""
    logger.info("Nhận lệnh /skip_photo")
    if 'clinical_data' not in context.user_data:
        await update.message.reply_text("⚠️ Vui lòng gửi dữ liệu lâm sàng trước!")
        return
    await update.message.reply_text("📸 Đã bỏ qua ảnh. Đang xử lý dữ liệu...")
    await process_data(update, context, image_path=None, clinical_data=context.user_data['clinical_data'])

async def skip_clinical(update: Update, context):
    """Cho phép bỏ qua dữ liệu lâm sàng và yêu cầu gửi ảnh."""
    logger.info("Nhận lệnh /skip_clinical")
    context.user_data.pop('clinical_data', None)  # Xóa dữ liệu lâm sàng nếu có
    await update.message.reply_text(
        "📋 Đã bỏ qua dữ liệu lâm sàng.\n"
        "📸 Vui lòng gửi hình ảnh để tiếp tục.",
        parse_mode='Markdown'
    )

async def handle_message(update: Update, context):
    """Xử lý tin nhắn JSON từ người dùng."""
    logger.info("Nhận tin nhắn văn bản")
    try:
        clinical_data = json.loads(update.message.text)
        errors = validate_clinical_data(clinical_data)
        if errors:
            await update.message.reply_text(f"⚠️ Lỗi dữ liệu:\n" + "\n".join(errors))
            return
        if not any(key in clinical_data for key in column_mapping):
            await update.message.reply_text("⚠️ JSON phải chứa ít nhất một chỉ số như age, blood_pressure, v.v.")
            return
        context.user_data['clinical_data'] = clinical_data
        await update.message.reply_text(
            "✅ Đã nhận dữ liệu lâm sàng.\n"
            "📸 Vui lòng gửi hình ảnh hoặc gửi `/skip_photo` để bỏ qua.",
            parse_mode='Markdown'
        )
        logger.info(f"Đã nhận dữ liệu lâm sàng: {clinical_data}")
    except json.JSONDecodeError:
        await update.message.reply_text("❌ Dữ liệu không đúng định dạng JSON. Vui lòng gửi lại.")
    except Exception as e:
        await update.message.reply_text(f"❌ Lỗi: {str(e)}")
        logger.error(f"Lỗi xử lý tin nhắn: {e}")

async def handle_photo(update: Update, context):
    """Xử lý ảnh từ người dùng, hỗ trợ trường hợp không có dữ liệu lâm sàng."""
    logger.info("Nhận ảnh")
    photo = update.message.photo[-1]
    file = await context.bot.get_file(photo.file_id)
    image_path = f"/tmp/{photo.file_id}.jpg"
    await file.download_to_drive(image_path)

    clinical_data = context.user_data.get('clinical_data', None)
    if clinical_data:
        await update.message.reply_text("📸 Đã nhận ảnh và dữ liệu lâm sàng. Đang xử lý...")
    else:
        await update.message.reply_text("📸 Đã nhận ảnh, dữ liệu lâm sàng đã bỏ qua. Đang xử lý chỉ với ảnh...")

    await process_data(update, context, image_path, clinical_data)

async def process_data(update: Update, context, image_path=None, clinical_data=None):
    """Xử lý dữ liệu lâm sàng và/hoặc ảnh, trả kết quả."""
    clinical_df = pd.DataFrame([clinical_data]) if clinical_data else pd.DataFrame()
    training_file_path = '/content/drive/MyDrive/TrainingWiDS2021_filled.csv'
    clinical_model_path = '/content/drive/MyDrive/main_clinical_model.pth'
    image_model_path = '/content/drive/MyDrive/working/meta_bestqwk'

    try:
        # Kiểm tra đầu vào
        if clinical_df.empty and not image_path:
            await update.message.reply_text("❌ Cần ít nhất dữ liệu lâm sàng hoặc ảnh để dự đoán!")
            return

        # Tải dữ liệu huấn luyện
        training_df = load_data(training_file_path, max_rows=5000)  # Giảm max_rows để tăng tốc

        # Xử lý dữ liệu lâm sàng nếu có
        if not clinical_df.empty:
            imputed_df = impute_missing_values(clinical_df, training_df)
            extended_df = predict_new_columns(training_df, imputed_df)
            X_processed, preprocessor = preprocess_clinical_data(extended_df)
        else:
            extended_df = pd.DataFrame()
            X_processed, preprocessor = preprocess_clinical_data(extended_df)

        # Dự đoán mức độ nghiêm trọng
        severity = predict_severity(
            extended_df if not extended_df.empty else None,
            image_path,
            clinical_model_path,
            image_model_path,
            preprocessor
        )

        # Chuẩn bị phản hồi
        severity_map = {0: "Rất nhẹ", 1: "Nhẹ", 2: "Trung bình", 3: "Nặng", 4: "Rất nặng"}
        severity_label = severity_map.get(severity[0], "Không xác định")
        response = (
            f"🎯 *Kết quả dự đoán*\n"
            f"🔹 Mức độ nghiêm trọng: *{severity[0]}* ({severity_label})\n"
            f"🔹 Dựa trên {'ảnh và dữ liệu lâm sàng' if not extended_df.empty else 'chỉ ảnh'}."
        )

        # Nếu có dữ liệu lâm sàng, hiển thị bảng bổ sung
        if not extended_df.empty:
            extended_df['predicted_severity'] = severity[0]
            table = "*Dữ liệu bổ sung*\n"
            table += "| Chỉ số | Giá trị |\n"
            table += "|---|---|\n"
            for col, val in extended_df.iloc[0].items():
                if pd.isna(val):
                    continue
                val_str = f"{val:.2f}" if isinstance(val, (int, float)) else str(val)
                table += f"| {col} | {val_str} |\n"
            response += f"\n\n{table}"

            # Lưu vào CSV
            output_path = '/content/drive/MyDrive/extended_clinical_data.csv'
            extended_df.to_csv(output_path, index=False)
            response += f"\n💾 Dữ liệu đã được lưu vào: `{output_path}`"

        response += "\n📋 Gửi JSON hoặc ảnh mới để tiếp tục!"
        await update.message.reply_text(response, parse_mode='Markdown')

    except Exception as e:
        await update.message.reply_text(f"❌ Dự đoán thất bại: {str(e)}")
        logger.error(f"Lỗi dự đoán: {e}")
    finally:
        context.user_data.clear()
        if image_path and os.path.exists(image_path):
            os.remove(image_path)

async def error_handler(update: Update, context):
    """Xử lý lỗi Telegram."""
    logger.warning(f'Update "{update}" gây lỗi "{context.error}"')

def main():
    """Chạy bot Telegram."""
    logger.info("Khởi động bot")
    os.environ['TZ'] = 'UTC'
    try:
        import time
        time.tzset()
        logger.info("Đặt múi giờ UTC")
    except AttributeError:
        logger.warning("Không thể đặt múi giờ")

    application = Application.builder().token(TELEGRAM_TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("skip_photo", skip_photo))
    application.add_handler(CommandHandler("skip_clinical", skip_clinical))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))
    application.add_error_handler(error_handler)
    application.run_polling(allowed_updates=Update.ALL_TYPES)

if __name__ == '__main__':
    main()

SyntaxError: unterminated string literal (detected at line 675) (<ipython-input-5-52fd608c1666>, line 675)

In [ ]:
!pip install nest_asyncio

In [ ]:
import os
import sqlite3
import logging
import atexit
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram import Update, Bot
import asyncio
import nest_asyncio
from fastapi import FastAPI, HTTPException, Depends
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel
from typing import Optional
import json
from telegram.error import InvalidToken
from datetime import datetime, timezone, timedelta
import pandas as pd
from google.colab import auth
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.colab import drive

# --- Khởi tạo môi trường ---
nest_asyncio.apply()

# Cấu hình logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Token chính và API
DEFAULT_TELEGRAM_TOKEN = "7604789951:AAGfAXF6mwzrajv8r_YDvIoQKiFc6rKHJ64"  # Thay bằng token hợp lệ
DOCTOR_API_TOKEN = "secure_doctor_token_123"  # Token cho API

# Đường dẫn cơ sở dữ liệu và CSV
DB_PATH = "/content/telegram_bot.db"
USERS_CSV_PATH = "/content/drive/MyDrive/users.csv"
INFO_CSV_PATH = "/content/drive/MyDrive/information.csv"

# Giới hạn số bot tối đa
MAX_BOTS = 10

# Cấu hình Google Drive API
SCOPES = ['https://www.googleapis.com/auth/drive.file']

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Google Drive API ---
def get_drive_service():
    """Kết nối với Google Drive API sử dụng xác thực Colab."""
    try:
        auth.authenticate_user()
        from google.auth import default
        creds, _ = default(scopes=SCOPES)
        return build('drive', 'v3', credentials=creds)
    except Exception as e:
        logger.error(f"Lỗi khi kết nối Google Drive: {e}")
        raise

def upload_csv_to_drive(file_path, file_name):
    """Lưu file CSV vào Google Drive."""
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        if os.path.exists(file_path):
            logger.info(f"Lưu file {file_name} vào {file_path}")
            return
        else:
            logger.error(f"File {file_path} không tồn tại")
            raise FileNotFoundError(f"File {file_path} không tồn tại")
    except Exception as e:
        logger.error(f"Lỗi khi lưu {file_name} vào Google Drive: {e}")
        raise

def export_to_csv():
    """Xuất dữ liệu từ SQLite sang CSV và lưu vào Google Drive."""
    try:
        conn = sqlite3.connect(DB_PATH)

        # Xuất bảng users
        users_df = pd.read_sql_query("SELECT * FROM users", conn)
        users_df.to_csv(USERS_CSV_PATH, index=False, encoding='utf-8')
        upload_csv_to_drive(USERS_CSV_PATH, 'users.csv')
        logger.info("Đã xuất và lưu users.csv vào Google Drive")

        # Xuất bảng information
        info_df = pd.read_sql_query("SELECT * FROM information", conn)
        info_df.to_csv(INFO_CSV_PATH, index=False, encoding='utf-8')
        upload_csv_to_drive(INFO_CSV_PATH, 'information.csv')
        logger.info("Đã xuất và lưu information.csv vào Google Drive")

        conn.close()
    except Exception as e:
        logger.error(f"Lỗi khi xuất dữ liệu sang CSV: {e}")

# --- Khởi tạo cơ sở dữ liệu SQLite ---
def init_db():
    """Khởi tạo cơ sở dữ liệu SQLite."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Bảng users
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                user_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                telegram_id TEXT,
                role TEXT NOT NULL CHECK(role IN ('doctor', 'patient')),
                telegram_token TEXT,
                bot_name TEXT,
                doctor_id INTEGER,
                FOREIGN KEY (doctor_id) REFERENCES users(user_id)
            )
        """)

        # Bảng information với các cột thời gian uống thuốc mới
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS information (
                user_id INTEGER PRIMARY KEY,
                tuoi INTEGER,
                benh TEXT,
                thuoc TEXT,
                sang TEXT,
                trua TEXT,
                toi TEXT,
                tai_kham TEXT,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        """)

        conn.commit()
        logger.info("Đã khởi tạo cơ sở dữ liệu SQLite")

        # Xuất dữ liệu ban đầu sang CSV
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi khởi tạo cơ sở dữ liệu: {e}")
        raise
    finally:
        conn.close()

# --- Xử lý cơ sở dữ liệu ---
async def insert_to_db(table: str, data: dict):
    """Chèn hoặc cập nhật dữ liệu vào SQLite và đồng bộ CSV."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        if table == "users":
            telegram_token = data.get('telegram_token')
            bot_name = data.get('bot_name', f"Bot_{data['user_id']}")
            role = data.get('role')
            if telegram_token and isinstance(telegram_token, str) and telegram_token.lower() != 'nan':
                try:
                    bot = Bot(token=telegram_token)
                    await bot.get_me()
                    logger.info(f"Token hợp lệ cho user_id {data['user_id']}: {telegram_token}")
                except Exception as e:
                    logger.error(f"Lỗi khi kiểm tra token cho user_id {data['user_id']}: {e}")
                    raise ValueError(f"Token Telegram không hợp lệ: {e}")

            cursor.execute("""
                INSERT OR REPLACE INTO users (user_id, name, telegram_id, role, telegram_token, bot_name, doctor_id)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data['name'],
                data.get('telegram_id'),
                role,
                telegram_token,
                bot_name,
                data.get('doctor_id') if role == 'patient' else None
            ))

        elif table == "information":
            cursor.execute("SELECT user_id, role FROM users WHERE user_id = ?", (data['user_id'],))
            user = cursor.fetchone()
            if not user:
                raise ValueError(f"Người dùng với ID {data['user_id']} không tồn tại")
            if user[1] != 'patient':
                raise ValueError(f"Chỉ bệnh nhân mới có thông tin chi tiết")

            cursor.execute("""
                INSERT OR REPLACE INTO information (user_id, tuoi, benh, thuoc, sang, trua, toi, tai_kham)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data.get('tuoi'),
                data.get('benh'),
                data.get('thuoc'),
                data.get('sang'),
                data.get('trua'),
                data.get('toi'),
                data.get('tai_kham')
            ))

        conn.commit()
        logger.info(f"Đã chèn/cập nhật dữ liệu vào bảng {table}: {data}")

        # Đồng bộ CSV sau khi cập nhật SQLite
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi chèn dữ liệu vào {table}: {e}")
        raise
    finally:
        conn.close()

def get_user(telegram_id: str):
    """Lấy thông tin người dùng từ SQLite dựa trên Telegram ID."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user_id, name, role, telegram_token, bot_name, doctor_id
            FROM users
            WHERE telegram_id = ?
        """, (telegram_id,))
        user = cursor.fetchone()
        conn.close()
        if user:
            return {
                'user_id': user[0],
                'name': user[1],
                'role': user[2],
                'telegram_token': user[3],
                'bot_name': user[4],
                'doctor_id': user[5]
            }
        return None
    except Exception as e:
        logger.error(f"Lỗi khi lấy người dùng với telegram_id {telegram_id}: {e}")
        return None

def get_user_by_id(user_id: int):
    """Lấy thông tin người dùng theo user_id."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user_id, name, role, telegram_token, bot_name, doctor_id
            FROM users
            WHERE user_id = ?
        """, (user_id,))
        user = cursor.fetchone()
        conn.close()
        if user:
            return {
                'user_id': user[0],
                'name': user[1],
                'role': user[2],
                'telegram_token': user[3],
                'bot_name': user[4],
                'doctor_id': user[5]
            }
        return None
    except Exception as e:
        logger.error(f"Lỗi khi lấy người dùng với user_id {user_id}: {e}")
        return None

def get_patients(doctor_id: int):
    """Lấy danh sách bệnh nhân của bác sĩ."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT u.user_id, u.name, u.telegram_id, u.telegram_token, u.bot_name, i.tuoi, i.benh, i.thuoc, i.sang, i.trua, i.toi, i.tai_kham
            FROM users u
            LEFT JOIN information i ON u.user_id = i.user_id
            WHERE u.doctor_id = ? AND u.role = 'patient'
        """, (doctor_id,))
        patients = cursor.fetchall()
        conn.close()
        return [{
            'user_id': p[0],
            'name': p[1],
            'telegram_id': p[2],
            'telegram_token': p[3],
            'bot_name': p[4],
            'tuoi': p[5],
            'benh': p[6],
            'thuoc': p[7],
            'sang': p[8],
            'trua': p[9],
            'toi': p[10],
            'tai_kham': p[11]
        } for p in patients]
    except Exception as e:
        logger.error(f"Lỗi khi lấy danh sách bệnh nhân cho bác sĩ {doctor_id}: {e}")
        return []

# --- Xử lý Telegram ---
bots = {}  # Lưu Application và Bot: {token: {'application': Application, 'bot': Bot}}

async def get_bot(telegram_token: str, is_private_bot: bool = False, is_doctor_bot: bool = False):
    """Tạo hoặc lấy Application instance cho bot."""
    global bots
    if not telegram_token or not isinstance(telegram_token, str) or telegram_token.lower() == 'nan':
        raise ValueError("Token không hợp lệ hoặc rỗng")

    if telegram_token not in bots:
        if len(bots) >= MAX_BOTS:
            raise ValueError(f"Đã đạt giới hạn {MAX_BOTS} bot!")
        try:
            temp_bot = Bot(token=telegram_token)
            await temp_bot.get_me()
            app = Application.builder().token(telegram_token).build()

            # Các lệnh chung
            app.add_handler(CommandHandler("start", start))
            app.add_handler(CommandHandler("get_id", get_id))
            app.add_handler(CommandHandler("cancel", cancel))

            # Lệnh cho bot chính hoặc bot riêng của bác sĩ
            if not is_private_bot or is_doctor_bot:
                app.add_handler(CommandHandler("cofirm", cofirm))
                app.add_handler(CommandHandler("add_doctor", add_doctor))
                app.add_handler(CommandHandler("add_patient", add_patient))
                app.add_handler(CommandHandler("list_patients", list_patients))
                app.add_handler(CommandHandler("update_token", update_token))
                app.add_handler(CommandHandler("register_bot", register_bot))
                app.add_handler(CommandHandler("list_bots", list_bots))

            # Lệnh cho bot riêng của bệnh nhân
            if is_private_bot and not is_doctor_bot:
                app.add_handler(CommandHandler("get_doctor_info", get_doctor_info))

            # Lệnh chỉ cho bot chính
            if not is_private_bot:
                app.add_handler(CommandHandler("start_bot", start_bot))
                app.add_handler(CommandHandler("stop_bot", stop_bot))
                app.add_handler(CommandHandler("delete_bot", delete_bot))

            app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
            app.add_error_handler(error_handler)

            await app.initialize()
            await app.start()
            await asyncio.sleep(1)
            await app.updater.start_polling(allowed_updates=Update.ALL_TYPES)
            bots[telegram_token] = {'application': app, 'bot': app.bot}
            logger.info(f"Bot {'riêng' if is_private_bot else 'chính'} với token {telegram_token} đã được khởi tạo")
            return app.bot
        except Exception as e:
            logger.error(f"Lỗi khi tạo bot với token {telegram_token}: {e}")
            raise ValueError(f"Lỗi khi tạo bot: {e}")

    return bots[telegram_token]['bot']

async def check_permission(update: Update, context, require_doctor: bool = False, main_bot_only: bool = False, confirm_access: bool = False, patient_access: bool = False):
    """Kiểm tra quyền truy cập của người dùng."""
    telegram_id = str(update.effective_user.id)
    bot_token = context.bot.token

    # Bot chính có toàn quyền
    if bot_token == DEFAULT_TELEGRAM_TOKEN:
        return True

    # Kiểm tra lệnh chỉ dành cho bot chính
    if main_bot_only:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bot chính mới có quyền thực hiện lệnh này!"
        )
        return False

    # Kiểm tra quyền xác nhận vai trò
    if confirm_access:
        user = get_user(telegram_id)
        if not user or user['role'] != 'doctor':
            await context.bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Chỉ bác sĩ mới có quyền xác nhận vai trò!"
            )
            return False
        return True

    user = get_user(telegram_id)
    if not user:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng xác nhận vai trò bằng /cofirm trước!"
        )
        return False

    if require_doctor and user['role'] != 'doctor':
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bác sĩ mới có quyền thực hiện lệnh này!"
        )
        return False

    if patient_access and user['role'] != 'patient':
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bệnh nhân mới có quyền thực hiện lệnh này!"
        )
        return False

    return True

async def start(update: Update, context):
    """Hiển thị thông báo chào và tự động xác nhận vai trò cho bot riêng."""
    logger.info(f"Nhận lệnh /start từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN
    is_private_bot = False
    is_doctor_bot = False

    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
            is_private_bot = True
            is_doctor_bot = user['role'] == 'doctor'
        except Exception as e:
            logger.error(f"Lỗi khi khởi động bot riêng: {e}")
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"⚠️ Lỗi khởi động bot riêng: {str(e)}"
            )

    if is_private_bot and user:
        context.args = [str(user['user_id']), telegram_id, user['role']]
        await cofirm(update, context)
        return

    commands = (
        "📋 Lệnh khả dụng:\n"
        "• /get_id - Lấy Telegram ID\n"
        "• /cofirm - Xác nhận vai trò (chỉ bác sĩ)\n"
        "• /cancel - Hủy thao tác\n"
    )
    welcome_message = f"👋 Chào bạn (ID: {telegram_id})!\nℹ️ Dùng /cofirm để xác nhận vai trò (yêu cầu bác sĩ).\n"

    if is_main_bot:
        commands = (
            "📋 Lệnh khả dụng:\n"
            "• /get_id - Lấy Telegram ID\n"
            "• /cofirm - Xác nhận vai trò\n"
            "• /add_doctor - Thêm bác sĩ\n"
            "• /add_patient - Thêm bệnh nhân\n"
            "• /list_patients - Xem bệnh nhân\n"
            "• /list_bots - Xem danh sách bot\n"
            "• /update_token - Cập nhật token\n"
            "• /register_bot - Đăng ký bot\n"
            "• /start_bot - Khởi động bot\n"
            "• /stop_bot - Dừng bot\n"
            "• /delete_bot - Xóa bot\n"
            "• /cancel - Hủy thao tác\n"
        )
        welcome_message = f"👋 Chào bạn (ID: {telegram_id})!\nℹ️ Đây là bot chính, bạn có toàn quyền!\n"
    elif user:
        if user['role'] == 'doctor':
            commands = (
                "📋 Lệnh khả dụng:\n"
                "• /get_id - Lấy Telegram ID\n"
                "• /cofirm - Xác nhận vai trò\n"
                "• /add_doctor - Thêm bác sĩ\n"
                "• /add_patient - Thêm bệnh nhân\n"
                "• /list_patients - Xem bệnh nhân\n"
                "• /list_bots - Xem danh sách bot\n"
                "• /update_token - Cập nhật token\n"
                "• /register_bot - Đăng ký bot\n"
                "• /cancel - Hủy thao tác\n"
            )
            welcome_message = f"👋 Chào bác sĩ {user['name']} (ID: {telegram_id})!\n"
        else:
            commands = (
                "📋 Lệnh khả dụng:\n"
                "• /get_id - Lấy Telegram ID\n"
                "• /get_doctor_info - Xem thông tin bác sĩ\n"
                "• /cancel - Hủy thao tác\n"
            )
            welcome_message = f"👋 Chào bệnh nhân {user['name']} (ID: {telegram_id})!\n"

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=welcome_message + commands + "🚀 Gửi lệnh để bắt đầu!"
    )

async def get_id(update: Update, context):
    """Lấy Telegram ID và hiển thị vai trò."""
    logger.info(f"Nhận lệnh /get_id từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    user = get_user(telegram_id)
    role = user['role'] if user else ("chính" if is_main_bot else "chưa xác nhận")
    name = user['name'] if user else "N/A"
    commands = (
        "• /get_id - Lấy Telegram ID\n"
        "• /cofirm - Xác nhận vai trò (chỉ bác sĩ)\n"
        "• /cancel - Hủy thao tác\n"
    )

    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
        except Exception as e:
            logger.error(f"Lỗi khi lấy bot riêng: {e}")

    if is_main_bot:
        commands = (
            "• /get_id - Lấy Telegram ID\n"
            "• /cofirm - Xác nhận vai trò\n"
            "• /add_doctor - Thêm bác sĩ\n"
            "• /add_patient - Thêm bệnh nhân\n"
            "• /list_patients - Xem bệnh nhân\n"
            "• /list_bots - Xem danh sách bot\n"
            "• /update_token - Cập nhật token\n"
            "• /register_bot - Đăng ký bot\n"
            "• /start_bot - Khởi động bot\n"
            "• /stop_bot - Dừng bot\n"
            "• /delete_bot - Xóa bot\n"
            "• /cancel - Hủy thao tác\n"
        )
    elif user and user['role'] == 'doctor':
        commands = (
            "• /get_id - Lấy Telegram ID\n"
            "• /cofirm - Xác nhận vai trò\n"
            "• /add_doctor - Thêm bác sĩ\n"
            "• /add_patient - Thêm bệnh nhân\n"
            "• /list_patients - Xem bệnh nhân\n"
            "• /list_bots - Xem danh sách bot\n"
            "• /update_token - Cập nhật token\n"
            "• /register_bot - Đăng ký bot\n"
            "• /cancel - Hủy thao tác\n"
        )
    elif user and user['role'] == 'patient':
        doctor = get_user_by_id(user['doctor_id']) if user['doctor_id'] else None
        doctor_info = (
            f"👨‍⚕️ Bác sĩ phụ trách: {doctor['name']} (ID: {doctor['user_id']})\n"
            if doctor else
            f"❌ Không tìm thấy bác sĩ phụ trách\n"
        )
        commands = (
            f"{doctor_info}"
            "📋 Lệnh khả dụng:\n"
            "• /get_id - Lấy Telegram ID\n"
            "• /get_doctor_info - Xem thông tin bác sĩ\n"
            "• /cancel - Hủy thao tác\n"
        )

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            f"📌 Telegram ID của bạn: {telegram_id}\n"
            f"👤 Vai trò: {role}\n"
            f"ℹ️ Tên: {name}\n"
            f"📋 Lệnh khả dụng:\n{commands}"
        )
    )

async def cofirm(update: Update, context):
    """Xác nhận vai trò người dùng."""
    logger.info(f"Nhận lệnh /cofirm từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    if not await check_permission(update, context, confirm_access=True):
        return

    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
        except Exception as e:
            logger.error(f"Lỗi khi lấy bot riêng: {e}")

    if user and not is_main_bot:
        commands = (
            "• /get_id - Lấy Telegram ID\n"
            "• /cofirm - Xác nhận vai trò\n"
            "• /add_doctor - Thêm bác sĩ\n"
            "• /add_patient - Thêm bệnh nhân\n"
            "• /list_patients - Xem bệnh nhân\n"
            "• /list_bots - Xem danh sách bot\n"
            "• /update_token - Cập nhật token\n"
            "• /register_bot - Đăng ký bot\n"
            "• /cancel - Hủy thao tác\n"
        ) if user['role'] == 'doctor' else (
            "• /get_id - Lấy Telegram ID\n"
            "• /get_doctor_info - Xem thông tin bác sĩ\n"
            "• /cancel - Hủy thao tác\n"
        )
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"✅ Bạn đã được xác nhận là {user['role']}: {user['name']} (ID: {telegram_id}).\n📋 Lệnh khả dụng:\n{commands}"
        )
        return

    if len(context.args) != 3:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 Xác nhận vai trò\n"
                "Vui lòng gửi: /cofirm <user_id> <telegram_id> <role>\n"
                "Ví dụ: /cofirm 101 123456789 patient\n"
                "Role: doctor hoặc patient\n"
                "❌ Gửi /cancel để hủy."
            )
        )
        return

    try:
        user_id = int(context.args[0])
        provided_telegram_id = str(context.args[1])
        role = context.args[2].lower()
        if role not in ['doctor', 'patient']:
            raise ValueError("Role phải là 'doctor' hoặc 'patient'")

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Kiểm tra telegram_id không trùng
        cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?", (provided_telegram_id, user_id))
        if cursor.fetchone():
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Telegram ID {provided_telegram_id} đã được liên kết với người dùng khác!"
            )
            conn.close()
            return

        cursor.execute("SELECT user_id, name, role, doctor_id, telegram_token FROM users WHERE user_id = ?", (user_id,))
        user_record = cursor.fetchone()
        if not user_record:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Không tìm thấy người dùng với ID: {user_id}"
            )
            conn.close()
            return

        if user_record[2] != role:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Vai trò {role} không khớp với vai trò đã đăng ký ({user_record[2]})!"
            )
            conn.close()
            return

        cursor.execute("UPDATE users SET telegram_id = ? WHERE user_id = ?", (provided_telegram_id, user_id))
        conn.commit()

        # Đồng bộ CSV
        export_to_csv()

        commands = (
            "• /get_id - Lấy Telegram ID\n"
            "• /cofirm - Xác nhận vai trò\n"
            "• /add_doctor - Thêm bác sĩ\n"
            "• /add_patient - Thêm bệnh nhân\n"
            "• /list_patients - Xem bệnh nhân\n"
            "• /list_bots - Xem danh sách bot\n"
            "• /update_token - Cập nhật token\n"
            "• /register_bot - Đăng ký bot\n"
            "• /cancel - Hủy thao tác\n"
        ) if role == 'doctor' else (
            "• /get_id - Lấy Telegram ID\n"
            "• /get_doctor_info - Xem thông tin bác sĩ\n"
            "• /cancel - Hủy thao tác\n"
        )
        doctor_info = ""
        if role == 'patient' and user_record[3]:
            doctor = get_user_by_id(user_record[3])
            doctor_info = (
                f"👨‍⚕️ Bác sĩ phụ trách: {doctor['name']} (ID: {doctor['user_id']})\n"
                if doctor else
                f"❌ Không tìm thấy bác sĩ phụ trách\n"
            )

        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                f"✅ Đã xác nhận vai trò {role}: {user_record[1]} (ID: {user_id}).\n"
                f"📌 Telegram ID liên kết: {provided_telegram_id}\n"
                f"{doctor_info}"
                f"📋 Lệnh khả dụng:\n{commands}"
            )
        )

        # Gửi hướng dẫn cho bệnh nhân qua bot riêng
        if role == 'patient' and user_record[4] and user_record[4] != DEFAULT_TELEGRAM_TOKEN:
            try:
                patient_bot = await get_bot(user_record[4], is_private_bot=True, is_doctor_bot=False)
                await patient_bot.send_message(
                    chat_id=provided_telegram_id,
                    text=(
                        f"👋 Chào bệnh nhân {user_record[1]} (ID: {user_id})!\n"
                        f"{doctor_info}"
                        f"📋 Lệnh khả dụng:\n"
                        "• /get_id - Lấy Telegram ID\n"
                        "• /get_doctor_info - Xem thông tin bác sĩ\n"
                        "• /cancel - Hủy thao tác\n"
                        "🚀 Bot riêng của bạn sẽ gửi nhắc nhở uống thuốc và tái khám!"
                    )
                )
                logger.info(f"Đã gửi hướng dẫn lệnh tới bệnh nhân {user_record[1]} qua bot riêng")
            except Exception as e:
                logger.error(f"Lỗi khi gửi hướng dẫn tới bệnh nhân {user_record[1]}: {e}")

        conn.close()
    except ValueError as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi: {str(e)}"
        )
    except Exception as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi không xác định: {str(e)}"
        )
        if 'conn' in locals():
            conn.close()

async def add_doctor(update: Update, context):
    """Trả về mẫu để thêm bác sĩ."""
    logger.info(f"Nhận lệnh /add_doctor từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    sample_data = {
        "user_id": 2,
        "name": "Dr. Hoa",
        "role": "doctor",
        "telegram_token": "your_bot_token_here",
        "bot_name": "Bot_2"
    }
    context.user_data['expecting'] = 'doctor'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 Thêm bác sĩ mới\n"
            f"Vui lòng gửi thông tin theo mẫu JSON:\n"
            f"```json\n{json.dumps(sample_data, indent=2, ensure_ascii=False)}\n```\n"
            "ℹ️ telegram_token và bot_name là tùy chọn.\n"
            "• Nếu không cung cấp bot_name, hệ thống gán Bot_<user_id>.\n"
            "• Nếu có telegram_token, bot riêng sẽ tự động chạy.\n"
            "• Bác sĩ cần dùng /cofirm để liên kết Telegram ID."
        )
    )

async def add_patient(update: Update, context):
    """Trả về mẫu để thêm bệnh nhân."""
    logger.info(f"Nhận lệnh /add_patient từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    sample_data = {
        "user_id": 101,
        "name": "Nguyễn Văn A",
        "role": "patient",
        "doctor_id": user['user_id'] if user else 1,
        "telegram_token": "your_bot_token_here",
        "information": {
            "tuoi": 65,
            "benh": "Tiểu đường",
            "thuoc": "Metformin",
            "sang": "07:30",
            "trua": "12:00",
            "toi": "18:00",
            "tai_kham": "2025-06-01"
        }
    }
    context.user_data['expecting'] = 'patient'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 Thêm bệnh nhân mới\n"
            f"Vui lòng gửi thông tin theo mẫu JSON:\n"
            f"```json\n{json.dumps(sample_data, indent=2, ensure_ascii=False)}\n```\n"
            "ℹ️ telegram_token, information, sang, trua, toi, tai_kham là tùy chọn.\n"
            "• Nếu có tai_kham, bot sẽ nhắc tái khám trước 2 ngày.\n"
            "• Chỉ cần điền ít nhất một trong sang, trua, toi.\n"
            "• Bệnh nhân cần dùng /cofirm để liên kết Telegram ID."
        )
    )

async def update_token(update: Update, context):
    """Cập nhật Telegram token riêng."""
    logger.info(f"Nhận lệnh /update_token từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    context.user_data['expecting'] = 'token'
    context.user_data['user_id'] = user['user_id'] if user else None
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 Cập nhật token bot riêng\n"
            "Vui lòng gửi Telegram token.\n"
            "❌ Gửi /cancel để hủy."
        )
    )

async def get_doctor_info(update: Update, context):
    """Lấy thông tin bác sĩ phụ trách."""
    logger.info(f"Nhận lệnh /get_doctor_info từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, patient_access=True):
        return

    telegram_id = str(update.effective_user.id)
    bot = context.bot
    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    doctor = get_user_by_id(user['doctor_id']) if user['doctor_id'] else None
    if not doctor:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Không tìm thấy thông tin bác sĩ phụ trách."
        )
        return

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "👨‍⚕️ Bác sĩ phụ trách:\n"
            f"• ID: {doctor['user_id']}\n"
            f"• Tên: {doctor['name']}\n"
            f"• Bot: {doctor['bot_name'] or 'Không có'}"
        )
    )

async def list_patients(update: Update, context):
    """Hiển thị danh sách bệnh nhân."""
    logger.info(f"Nhận lệnh /list_patients từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    patients = get_patients(user['user_id'] if user else 1)
    if not patients:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Bạn chưa có bệnh nhân nào."
        )
        return

    message = f"📋 Danh sách bệnh nhân của bác sĩ {user['name'] if user else 'N/A'} (ID: {user['user_id'] if user else 'N/A'}):\n\n"
    for p in patients:
        message += (
            f"🧑 Bệnh nhân {p['name']} (ID: {p['user_id']}):\n"
            f"• Tuổi: {p['tuoi'] or 'N/A'}\n"
            f"• Bệnh: {p['benh'] or 'N/A'}\n"
            f"• Thuốc: {p['thuoc'] or 'N/A'}\n"
            f"• Nhắc nhở uống thuốc:\n"
            f"  - Sáng: {p['sang'] or 'N/A'}\n"
            f"  - Trưa: {p['trua'] or 'N/A'}\n"
            f"  - Tối: {p['toi'] or 'N/A'}\n"
            f"• Tái khám: {p['tai_kham'] or 'N/A'}\n"
            f"• Telegram ID: {p['telegram_id'] or 'Chưa xác nhận'}\n"
            f"• Bot riêng: {'Có' if p['telegram_token'] else 'Không'}\n"
            f"---\n"
        )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message
    )

async def list_bots(update: Update, context):
    """Liệt kê tất cả bot riêng."""
    logger.info(f"Nhận lệnh /list_bots từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_id, telegram_token, name, bot_name, role FROM users WHERE telegram_token IS NOT NULL")
    bot_mappings = cursor.fetchall()
    conn.close()

    if not bot_mappings:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Chưa có bot riêng nào được thiết lập."
        )
        return

    message = "🤖 Danh sách bot riêng:\n\n"
    for mapping in bot_mappings:
        telegram_id, telegram_token, name, bot_name, role = mapping
        if telegram_token and telegram_token.lower() != 'nan':
            status = "✅ Đang chạy" if telegram_token in bots and bots[telegram_token]['application'].updater.running else "❌ Không chạy"
            message += (
                f"{'👨‍⚕️' if role == 'doctor' else '🧑'} {role.capitalize()}: {name}\n"
                f"🆔 Telegram ID: {telegram_id or 'Chưa xác nhận'}\n"
                f"🤖 Bot: {bot_name}\n"
                f"📊 Trạng thái: {status}\n"
                f"---\n"
            )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message
    )

async def start_bot(update: Update, context):
    """Khởi động bot riêng."""
    logger.info(f"Nhận lệnh /start_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp bot_name. Ví dụ: /start_bot Bot_5"
        )
        return

    bot_name = context.args[0]
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token, role FROM users WHERE bot_name = ?", (bot_name,))
    result = cursor.fetchone()
    conn.close()

    if not result:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Không tìm thấy bot với bot_name: {bot_name}"
        )
        return

    telegram_token, role = result
    if not telegram_token or telegram_token.lower() == 'nan':
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Bot {bot_name} không có token hợp lệ. Cập nhật bằng /update_token."
        )
        return

    if telegram_token in bots and bots[telegram_token]['application'].updater.running:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"ℹ️ Bot {bot_name} đã đang chạy!"
        )
        return

    try:
        await get_bot(telegram_token, is_private_bot=True, is_doctor_bot=role == 'doctor')
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"✅ Đã khởi động bot: {bot_name}"
        )
    except Exception as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi khi khởi động bot {bot_name}: {str(e)}"
        )

async def stop_bot(update: Update, context):
    """Dừng bot riêng."""
    logger.info(f"Nhận lệnh /stop_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp bot_name. Ví dụ: /stop_bot Bot_5"
        )
        return

    bot_name = context.args[0]
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token FROM users WHERE bot_name = ?", (bot_name,))
    result = cursor.fetchone()
    conn.close()

    if not result:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Không tìm thấy bot với bot_name: {bot_name}"
        )
        return

    telegram_token = result[0]
    if telegram_token in bots:
        try:
            await bots[telegram_token]['application'].updater.stop()
            await bots[telegram_token]['application'].stop()
            await bots[telegram_token]['application'].shutdown()
            del bots[telegram_token]
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"✅ Đã dừng bot: {bot_name}"
            )
        except Exception as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi khi dừng bot {bot_name}: {str(e)}"
            )
    else:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"ℹ️ Bot {bot_name} không đang chạy."
        )

async def delete_bot(update: Update, context):
    """Xóa bot và thông tin người dùng."""
    logger.info(f"Nhận lệnh /delete_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp bot_name. Ví dụ: /delete_bot Bot_5"
        )
        return

    bot_name = context.args[0]
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token, user_id, role FROM users WHERE bot_name = ?", (bot_name,))
    result = cursor.fetchone()

    if not result:
        conn.close()
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Không tìm thấy bot với bot_name: {bot_name}"
        )
        return

    telegram_token, user_id, role = result
    try:
        if telegram_token and telegram_token in bots:
            await bots[telegram_token]['application'].updater.stop()
            await bots[telegram_token]['application'].stop()
            await bots[telegram_token]['application'].shutdown()
            del bots[telegram_token]

        if role == 'patient':
            cursor.execute("DELETE FROM information WHERE user_id = ?", (user_id,))
        cursor.execute("DELETE FROM users WHERE user_id = ?", (user_id,))
        conn.commit()

        # Đồng bộ CSV
        export_to_csv()

        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"✅ Đã xóa bot: {bot_name} và thông tin {role} (ID: {user_id})."
        )
    except Exception as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi khi xóa bot {bot_name}: {str(e)}"
        )
    finally:
        conn.close()

async def cancel(update: Update, context):
    """Hủy thao tác đang chờ."""
    logger.info(f"Nhận lệnh /cancel từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    context.user_data.pop('expecting', None)
    context.user_data.pop('user_id', None)
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text="❌ Đã hủy thao tác."
    )

async def register_bot(update: Update, context):
    """Đăng ký bot riêng."""
    logger.info(f"Nhận lệnh /register_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'register_bot'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 Đăng ký bot riêng\n"
            "Vui lòng gửi thông tin theo định dạng:\n"
            "Tên: <tên bác sĩ>\n"
            "Token: <telegram_token>\n"
            "Bot Name: <tên bot tùy chọn>\n\n"
            "Ví dụ:\n"
            "Tên: Dr. Minh\n"
            "Token: 1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p\n"
            "Bot Name: Bot_3\n\n"
            "❌ Gửi /cancel để hủy."
        )
    )

async def send_reminders():
    """Gửi thông báo uống thuốc và tái khám cho bệnh nhân."""
    logger.info("Kiểm tra lịch nhắc nhở")
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT u.user_id, u.name, u.telegram_id, u.telegram_token, u.doctor_id, i.thuoc, i.sang, i.trua, i.toi, i.tai_kham
            FROM users u
            JOIN information i ON u.user_id = i.user_id
            WHERE u.role = 'patient'
        """)
        patients = cursor.fetchall()
        conn.close()

        current_time = datetime.now(timezone(timedelta(hours=7))).strftime("%H:%M")
        current_date = datetime.now(timezone(timedelta(hours=7))).date()
        reminder_date = (current_date + timedelta(days=2)).strftime("%Y-%m-%d")

        for patient in patients:
            user_id, name, telegram_id, telegram_token, doctor_id, thuoc, sang, trua, toi, tai_kham = patient
            try:
                target_bot = None
                if telegram_token and telegram_token.lower() != 'nan':
                    target_bot = await get_bot(telegram_token, is_private_bot=True, is_doctor_bot=False)
                else:
                    doctor = get_user_by_id(doctor_id)
                    if doctor and doctor['telegram_token'] and doctor['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
                        target_bot = await get_bot(doctor['telegram_token'], is_private_bot=True, is_doctor_bot=True)
                    else:
                        target_bot = await get_bot(DEFAULT_TELEGRAM_TOKEN)

                if telegram_id:
                    # Nhắc uống thuốc
                    for time_slot, label in [(sang, "Sáng"), (trua, "Trưa"), (toi, "Tối")]:
                        if time_slot and time_slot == current_time:
                            await target_bot.send_message(
                                chat_id=telegram_id,
                                text=(
                                    f"⏰ Nhắc nhở uống thuốc ({label}):\n"
                                    f"• Bệnh nhân: {name}\n"
                                    f"• Thuốc: {thuoc}\n"
                                    f"• Thời gian: {time_slot}"
                                )
                            )
                            logger.info(f"Đã gửi nhắc nhở uống thuốc ({label}) cho bệnh nhân {name} (ID: {user_id})")

                    # Nhắc tái khám
                    if tai_kham and tai_kham == reminder_date:
                        await target_bot.send_message(
                            chat_id=telegram_id,
                            text=(
                                f"🩺 Nhắc nhở tái khám:\n"
                                f"• Bệnh nhân: {name}\n"
                                f"• Ngày tái khám: {tai_kham}\n"
                                f"• Vui lòng liên hệ bác sĩ để đặt lịch!"
                            )
                        )
                        logger.info(f"Đã gửi nhắc nhở tái khám cho bệnh nhân {name} (ID: {user_id})")
            except Exception as e:
                logger.error(f"Lỗi khi gửi nhắc nhở cho bệnh nhân {name} (ID: {user_id}): {e}")
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra lịch nhắc nhở: {e}")

def schedule_reminders():
    """Lên lịch kiểm tra thông báo mỗi phút."""
    loop = asyncio.get_event_loop()
    async def periodic():
        while True:
            await send_reminders()
            await asyncio.sleep(60)
    loop.create_task(periodic())

async def handle_message(update: Update, context):
    """Xử lý tin nhắn từ người dùng."""
    logger.info(f"Nhận tin nhắn từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    expecting = context.user_data.get('expecting')
    if not expecting:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Vui lòng dùng /add_doctor, /add_patient, /update_token, /register_bot trước!"
        )
        return

    if expecting == 'token':
        token = update.message.text.strip()
        user_id = context.user_data.get('user_id')
        try:
            is_doctor_token = user and user['role'] == 'doctor'
            bot_instance = await get_bot(token, is_private_bot=True, is_doctor_bot=is_doctor_token)
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            if not user_id:
                cursor.execute("SELECT MAX(user_id) FROM users")
                max_id = cursor.fetchone()[0] or 0
                new_id = max_id + 1
                bot_name = f"Bot_{new_id}"
                cursor.execute("""
                    INSERT INTO users (user_id, name, telegram_id, role, telegram_token, bot_name)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (new_id, f"Doctor_{new_id}", telegram_id, 'doctor', token, bot_name))
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 Đã tạo bác sĩ mới:\n"
                        f"• ID: {new_id}\n"
                        f"• Tên: Doctor_{new_id}\n"
                        f"• Bot: {bot_name}\n"
                        f"🤖 Bot riêng của bạn đang chạy."
                    )
                )
            else:
                cursor.execute("UPDATE users SET telegram_token = ? WHERE user_id = ?", (token, user_id))
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 Đã cập nhật token cho {'bác sĩ' if is_doctor_token else 'bệnh nhân'} (ID: {user_id}).\n"
                        f"🤖 Bot riêng của bạn đang chạy."
                    )
                )
            conn.commit()

            # Đồng bộ CSV
            export_to_csv()

            conn.close()
            context.user_data.pop('expecting', None)
            context.user_data.pop('user_id', None)
        except Exception as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}\nℹ️ Kiểm tra token hoặc tạo mới qua @BotFather."
            )
        return

    if expecting == 'register_bot':
        try:
            lines = update.message.text.strip().split('\n')
            data = {}
            for line in lines:
                if line.startswith('Tên:'):
                    data['name'] = line.replace('Tên:', '').strip()
                elif line.startswith('Token:'):
                    data['telegram_token'] = line.replace('Token:', '').strip()
                elif line.startswith('Bot Name:'):
                    data['bot_name'] = line.replace('Bot Name:', '').strip()

            if not data.get('name') or not data.get('telegram_token'):
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text="❌ Thiếu Tên hoặc Token. Vui lòng gửi đúng định dạng!"
                )
                return

            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(user_id) FROM users")

            max_id = cursor.fetchone()[0] or 0
            data['user_id'] = max_id + 1
            data['role'] = 'doctor'
            if not data.get('bot_name'):
                data['bot_name'] = f"Bot_{data['user_id']}"

            cursor.execute("SELECT bot_name FROM users WHERE bot_name = ?", (data['bot_name'],))
            if cursor.fetchone():
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=f"❌ Tên bot {data['bot_name']} đã tồn tại. Vui lòng chọn tên khác!"
                )
                conn.close()
                return

            try:
                bot_instance = await get_bot(data['telegram_token'], is_private_bot=True, is_doctor_bot=True)
                bot_info = await bot_instance.get_me()
                logger.info(f"Đã xác minh bot riêng: {bot_info.username}, ID: {bot_info.id}")
            except Exception as e:
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=f"❌ Token không hợp lệ: {str(e)}. Vui lòng kiểm tra hoặc tạo mới qua @BotFather."
                )
                conn.close()
                return

            await insert_to_db('users', data)
            conn.commit()

            # Đồng bộ CSV
            export_to_csv()

            conn.close()

            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=(
                    f"🎉 Đã đăng ký bot riêng thành công:\n"
                    f"• ID: {data['user_id']}\n"
                    f"• Tên: {data['name']}\n"
                    f"• Bot: {data['bot_name']}\n"
                    f"🤖 Bot riêng của bạn đang chạy.\n"
                    f"ℹ️ Dùng /cofirm để liên kết Telegram ID."
                )
            )
            context.user_data.pop('expecting', None)
        except Exception as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi khi đăng ký bot: {str(e)}"
            )
            if 'conn' in locals():
                conn.close()
        return

    if expecting in ['doctor', 'patient']:
        try:
            json_data = json.loads(update.message.text)
            if expecting == 'doctor':
                if not isinstance(json_data.get('user_id'), int) or not json_data.get('name') or json_data.get('role') != 'doctor':
                    raise ValueError("Thiếu hoặc sai định dạng user_id (số nguyên), name, hoặc role (phải là doctor)")
                if 'telegram_id' in json_data and json_data['telegram_id']:
                    json_data['telegram_id'] = str(json_data['telegram_id'])
                    conn = sqlite3.connect(DB_PATH)
                    cursor = conn.cursor()
                    cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?", (json_data['telegram_id'], json_data['user_id']))
                    if cursor.fetchone():
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"❌ Telegram ID {json_data['telegram_id']} đã được liên kết với người dùng khác!"
                        )
                        conn.close()
                        return
                    conn.close()
                if json_data.get('telegram_token'):
                    try:
                        bot_instance = await get_bot(json_data['telegram_token'], is_private_bot=True, is_doctor_bot=True)
                        bot_info = await bot_instance.get_me()
                        logger.info(f"Đã xác minh bot riêng cho bác sĩ: {bot_info.username}, ID: {bot_info.id}")
                    except Exception as e:
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"❌ Token không hợp lệ: {str(e)}. Bác sĩ đã được thêm nhưng bot riêng không chạy."
                        )
                        json_data['telegram_token'] = None
                await insert_to_db('users', json_data)
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"✅ Đã thêm bác sĩ:\n"
                        f"• ID: {json_data['user_id']}\n"
                        f"• Tên: {json_data['name']}\n"
                        f"• Bot: {json_data.get('bot_name', 'Bot_' + str(json_data['user_id']))}\n"
                        f"ℹ️ Bác sĩ cần dùng /cofirm để liên kết Telegram ID."
                    )
                )
            else:  # patient
                required_fields = ['user_id', 'name', 'doctor_id']
                if not all(isinstance(json_data.get(field), (int, str)) for field in required_fields) or json_data.get('role') != 'patient':
                    raise ValueError("Thiếu hoặc sai định dạng user_id, name, doctor_id, hoặc role (phải là patient)")
                if 'telegram_id' in json_data and json_data['telegram_id']:
                    json_data['telegram_id'] = str(json_data['telegram_id'])
                    conn = sqlite3.connect(DB_PATH)
                    cursor = conn.cursor()
                    cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?", (json_data['telegram_id'], json_data['user_id']))
                    if cursor.fetchone():
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"❌ Telegram ID {json_data['telegram_id']} đã được liên kết với người dùng khác!"
                        )
                        conn.close()
                        return
                    conn.close()
                if json_data.get('telegram_token'):
                    try:
                        bot_instance = await get_bot(json_data['telegram_token'], is_private_bot=True, is_doctor_bot=False)
                        bot_info = await bot_instance.get_me()
                        logger.info(f"Đã xác minh bot riêng cho bệnh nhân: {bot_info.username}, ID: {bot_info.id}")
                    except Exception as e:
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"❌ Token không hợp lệ: {str(e)}. Bệnh nhân đã được thêm nhưng bot riêng không chạy."
                        )
                        json_data['telegram_token'] = None
                await insert_to_db('users', json_data)
                if json_data.get('information'):
                    json_data['information']['user_id'] = json_data['user_id']
                    await insert_to_db('information', json_data['information'])
                doctor = get_user_by_id(json_data['doctor_id'])
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"✅ Đã thêm bệnh nhân:\n"
                        f"• ID: {json_data['user_id']}\n"
                        f"• Tên: {json_data['name']}\n"
                        f"• Bác sĩ phụ trách: {doctor['name'] if doctor else 'N/A'} (ID: {json_data['doctor_id']})\n"
                        f"ℹ️ Bệnh nhân cần dùng /cofirm để liên kết Telegram ID."
                    )
                )
            context.user_data.pop('expecting', None)
        except json.JSONDecodeError:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Tin nhắn phải là JSON hợp lệ. Vui lòng kiểm tra lại!"
            )
        except ValueError as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}"
            )
        except Exception as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi không xác định: {str(e)}"
            )
        return

async def error_handler(update: Update, context):
    """Xử lý lỗi."""
    logger.error(f"Cập nhật {update} gây ra lỗi: {context.error}")
    try:
        if update and update.effective_chat:
            await context.bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Đã xảy ra lỗi: {str(context.error)}. Vui lòng thử lại!"
            )
    except Exception as e:
        logger.error(f"Lỗi khi gửi thông báo lỗi: {e}")

# --- FastAPI ---
app = FastAPI()
security = HTTPBearer()

class DoctorData(BaseModel):
    user_id: int
    name: str
    role: str = "doctor"
    telegram_id: Optional[str] = None
    telegram_token: Optional[str] = None
    bot_name: Optional[str] = None

async def verify_doctor_token(credentials: HTTPAuthorizationCredentials = Depends(security)):
    if credentials.credentials != DOCTOR_API_TOKEN:
        raise HTTPException(status_code=401, detail="Token không hợp lệ")
    return credentials

@app.post("/add_doctor")
async def api_add_doctor(data: DoctorData, token: str = Depends(verify_doctor_token)):
    try:
        await insert_to_db('users', data.dict())
        return {"message": f"Đã thêm bác sĩ: {data.name}"}
    except Exception as e:
        logger.error(f"Lỗi khi thêm bác sĩ qua API: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# --- Khởi động bot chính và lên lịch nhắc nhở ---
async def main():
    global bots
    try:
        init_db()
        bot = await get_bot(DEFAULT_TELEGRAM_TOKEN)
        schedule_reminders()
        logger.info("Bot chính đã khởi động")
    except Exception as e:
        logger.error(f"Lỗi khi khởi động bot chính: {e}")
        raise

async def shutdown():
    """Dừng tất cả bot và đồng bộ dữ liệu trước khi thoát."""
    try:
        # Đồng bộ dữ liệu lần cuối
        export_to_csv()
        logger.info("Đã đồng bộ dữ liệu sang CSV trước khi tắt")

        # Dừng tất cả bot
        for token, bot_info in list(bots.items()):
            try:
                await bot_info['application'].updater.stop()
                await bot_info['application'].stop()
                await bot_info['application'].shutdown()
                logger.info(f"Đã dừng bot với token {token}")
            except Exception as e:
                logger.error(f"Lỗi khi dừng bot {token}: {e}")
            finally:
                bots.pop(token, None)
    except Exception as e:
        logger.error(f"Lỗi khi tắt bot: {e}")

atexit.register(lambda: asyncio.run(shutdown()))

if __name__ == "__main__":
    import uvicorn
    loop = asyncio.get_event_loop()
    loop.create_task(main())
    uvicorn.run(app, host="0.0.0.0", port=8000)

ModuleNotFoundError: No module named 'telegram'

In [ ]:
!pip install python-telegram-bot --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install aiofiles

In [ ]:
!pip install fastapi uvicorn python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
 !pip install aiofiles

In [ ]:
import os
import sqlite3
import logging
import atexit
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram import Update, Bot
import asyncio
import nest_asyncio
from fastapi import FastAPI, HTTPException, Depends
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel
from typing import Optional
import json
from telegram.error import InvalidToken
from datetime import datetime, timezone, timedelta
import pandas as pd
from google.colab import auth
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.colab import drive

# --- Khởi tạo môi trường ---
nest_asyncio.apply()

# Cấu hình logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Token và API
DEFAULT_TELEGRAM_TOKEN = "7604789951:AAGfAXF6mwzrajv8r_YDvIoQKiFc6rKHJ64"
DOCTOR_API_TOKEN = "secure_doctor_token_123"

# Đường dẫn tệp
DB_PATH = "/content/telegram_bot.db"
USERS_CSV_PATH = "/content/drive/MyDrive/users.csv"
INFO_CSV_PATH = "/content/drive/MyDrive/information.csv"
DOCTORS_LOG_CSV_PATH = "/content/drive/MyDrive/doctors_log.csv"

# Giới hạn bot
MAX_BOTS = 10

# Cấu hình Google Drive API
SCOPES = ['https://www.googleapis.com/auth/drive.file']

# Mount Google Drive
drive.mount('/content/drive')

# --- Google Drive API ---
def get_drive_service():
    try:
        auth.authenticate_user()
        from google.auth import default
        creds, _ = default(scopes=SCOPES)
        return build('drive', 'v3', credentials=creds)
    except Exception as e:
        logger.error(f"Lỗi khi kết nối Google Drive: {e}")
        raise

def upload_csv_to_drive(file_path, file_name):
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        if os.path.exists(file_path):
            logger.info(f"Lưu file {file_name} vào {file_path}")
            return
        else:
            logger.error(f"File {file_path} không tồn tại")
            raise FileNotFoundError(f"File {file_path} không tồn tại")
    except Exception as e:
        logger.error(f"Lỗi khi lưu {file_name} vào Google Drive: {e}")
        raise

def export_to_csv():
    try:
        conn = sqlite3.connect(DB_PATH)

        # Xuất bảng users
        users_df = pd.read_sql_query("SELECT * FROM users", conn)
        users_df.to_csv(USERS_CSV_PATH, index=False, encoding='utf-8')
        upload_csv_to_drive(USERS_CSV_PATH, 'users.csv')
        logger.info("Đã xuất users.csv vào Google Drive")

        # Xuất bảng information
        info_df = pd.read_sql_query("SELECT * FROM information", conn)
        info_df.to_csv(INFO_CSV_PATH, index=False, encoding='utf-8')
        upload_csv_to_drive(INFO_CSV_PATH, 'information.csv')
        logger.info("Đã xuất information.csv vào Google Drive")

        # Xuất bảng doctors_log
        doctors_log_df = pd.read_sql_query("SELECT * FROM doctors_log", conn)
        doctors_log_df.to_csv(DOCTORS_LOG_CSV_PATH, index=False, encoding='utf-8')
        upload_csv_to_drive(DOCTORS_LOG_CSV_PATH, 'doctors_log.csv')
        logger.info("Đã xuất doctors_log.csv vào Google Drive")

        conn.close()
    except Exception as e:
        logger.error(f"Lỗi khi xuất dữ liệu sang CSV: {e}")

# --- Khởi tạo cơ sở dữ liệu ---
def init_db():
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Bảng users
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                user_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                telegram_id TEXT,
                role TEXT NOT NULL CHECK(role IN ('doctor', 'patient')),
                telegram_token TEXT,
                bot_name TEXT,
                doctor_id INTEGER,
                FOREIGN KEY (doctor_id) REFERENCES users(user_id)
            )
        """)

        # Bảng information
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS information (
                user_id INTEGER PRIMARY KEY,
                tuoi INTEGER,
                benh TEXT,
                thuoc TEXT,
                sang TEXT,
                trua TEXT,
                toi TEXT,
                tai_kham TEXT,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        """)

        # Bảng doctors_log
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS doctors_log (
                user_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                telegram_id TEXT,
                address TEXT,
                phone TEXT,
                specialty TEXT,
                updated_at TEXT,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        """)

        conn.commit()
        logger.info("Đã khởi tạo cơ sở dữ liệu SQLite")
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi khởi tạo cơ sở dữ liệu: {e}")
        raise
    finally:
        conn.close()

# --- Xử lý cơ sở dữ liệu ---
async def insert_to_db(table: str, data: dict):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        if table == "users":
            telegram_token = data.get('telegram_token')
            bot_name = data.get('bot_name', f"Bot_{data['user_id']}")
            role = data.get('role')
            if telegram_token and isinstance(telegram_token, str) and telegram_token.lower() != 'nan':
                try:
                    bot = Bot(token=telegram_token)
                    await bot.get_me()
                    logger.info(f"Token hợp lệ cho user_id {data['user_id']}")
                except Exception as e:
                    logger.error(f"Lỗi khi kiểm tra token cho user_id {data['user_id']}: {e}")
                    raise ValueError(f"Token Telegram không hợp lệ: {e}")

            cursor.execute("""
                INSERT OR REPLACE INTO users (user_id, name, telegram_id, role, telegram_token, bot_name, doctor_id)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data['name'],
                data.get('telegram_id'),
                role,
                telegram_token,
                bot_name,
                data.get('doctor_id') if role == 'patient' else None
            ))

        elif table == "information":
            cursor.execute("SELECT user_id, role FROM users WHERE user_id = ?", (data['user_id'],))
            user = cursor.fetchone()
            if not user:
                raise ValueError(f"Người dùng với ID {data['user_id']} không tồn tại")
            if user[1] != 'patient':
                raise ValueError(f"Chỉ bệnh nhân mới có thông tin chi tiết")

            cursor.execute("""
                INSERT OR REPLACE INTO information (user_id, tuoi, benh, thuoc, sang, trua, toi, tai_kham)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data.get('tuoi'),
                data.get('benh'),
                data.get('thuoc'),
                data.get('sang'),
                data.get('trua'),
                data.get('toi'),
                data.get('tai_kham')
            ))

        elif table == "doctors_log":
            cursor.execute("SELECT user_id, role FROM users WHERE user_id = ?", (data['user_id'],))
            user = cursor.fetchone()
            if not user or user[1] != 'doctor':
                raise ValueError(f"Chỉ bác sĩ mới có thể cập nhật thông tin log")
            cursor.execute("""
                INSERT OR REPLACE INTO doctors_log (user_id, name, telegram_id, address, phone, specialty, updated_at)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data['user_id'],
                data['name'],
                data.get('telegram_id'),
                data.get('address'),
                data.get('phone'),
                data.get('specialty'),
                datetime.now(timezone(timedelta(hours=7))).strftime("%Y-%m-%d %H:%M:%S")
            ))

        conn.commit()
        logger.info(f"Đã chèn/cập nhật dữ liệu vào bảng {table}: {data}")
        export_to_csv()
    except Exception as e:
        logger.error(f"Lỗi khi chèn dữ liệu vào {table}: {e}")
        raise
    finally:
        conn.close()

def get_user(telegram_id: str):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user_id, name, role, telegram_token, bot_name, doctor_id
            FROM users
            WHERE telegram_id = ?
        """, (telegram_id,))
        user = cursor.fetchone()
        conn.close()
        if user:
            return {
                'user_id': user[0],
                'name': user[1],
                'role': user[2],
                'telegram_token': user[3],
                'bot_name': user[4],
                'doctor_id': user[5]
            }
        return None
    except Exception as e:
        logger.error(f"Lỗi khi lấy người dùng với telegram_id {telegram_id}: {e}")
        return None

def get_user_by_id(user_id: int):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user_id, name, role, telegram_token, bot_name, doctor_id
            FROM users
            WHERE user_id = ?
        """, (user_id,))
        user = cursor.fetchone()
        conn.close()
        if user:
            return {
                'user_id': user[0],
                'name': user[1],
                'role': user[2],
                'telegram_token': user[3],
                'bot_name': user[4],
                'doctor_id': user[5]
            }
        return None
    except Exception as e:
        logger.error(f"Lỗi khi lấy người dùng với user_id {user_id}: {e}")
        return None

def get_patients(doctor_id: int):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT u.user_id, u.name, u.telegram_id, u.telegram_token, u.bot_name, i.tuoi, i.benh, i.thuoc, i.sang, i.trua, i.toi, i.tai_kham
            FROM users u
            LEFT JOIN information i ON u.user_id = i.user_id
            WHERE u.doctor_id = ? AND u.role = 'patient'
        """, (doctor_id,))
        patients = cursor.fetchall()
        conn.close()
        return [{
            'user_id': p[0],
            'name': p[1],
            'telegram_id': p[2],
            'telegram_token': p[3],
            'bot_name': p[4],
            'tuoi': p[5],
            'benh': p[6],
            'thuoc': p[7],
            'sang': p[8],
            'trua': p[9],
            'toi': p[10],
            'tai_kham': p[11]
        } for p in patients]
    except Exception as e:
        logger.error(f"Lỗi khi lấy danh sách bệnh nhân cho bác sĩ {doctor_id}: {e}")
        return []

def get_doctor_log(doctor_id: int):
    try:
        df = pd.read_csv(DOCTORS_LOG_CSV_PATH)
        doctor_log = df[df['user_id'] == doctor_id].to_dict('records')
        return doctor_log[0] if doctor_log else None
    except Exception as e:
        logger.error(f"Lỗi khi lấy thông tin bác sĩ từ CSV: {e}")
        return None

# --- Hàm hỗ trợ nhắc nhở ---
def is_time_match(time_slot: str, current_time: str) -> bool:
    if not time_slot:
        return False
    try:
        slot_time = datetime.strptime(time_slot, "%H:%M")
        curr_time = datetime.strptime(current_time, "%H:%M")
        return abs((slot_time - curr_time).total_seconds()) <= 60
    except ValueError:
        return False

# --- Xử lý Telegram ---
bots = {}

async def get_bot(telegram_token: str, is_private_bot: bool = False, is_doctor_bot: bool = False):
    global bots
    if not telegram_token or not isinstance(telegram_token, str) or telegram_token.lower() == 'nan':
        raise ValueError("Token không hợp lệ hoặc rỗng")

    if telegram_token not in bots:
        if len(bots) >= MAX_BOTS:
            raise ValueError(f"Đã đạt giới hạn {MAX_BOTS} bot!")
        try:
            temp_bot = Bot(token=telegram_token)
            await temp_bot.get_me()
            app = Application.builder().token(telegram_token).build()

            # Lệnh chung
            app.add_handler(CommandHandler("start", start))
            app.add_handler(CommandHandler("get_id", get_id))
            app.add_handler(CommandHandler("cancel", cancel))

            # Lệnh cho bot chính hoặc bot riêng của bác sĩ
            if not is_private_bot or is_doctor_bot:
                app.add_handler(CommandHandler("confirm", confirm))
                app.add_handler(CommandHandler("add_doctor", add_doctor))
                app.add_handler(CommandHandler("add_patient", add_patient))
                app.add_handler(CommandHandler("list_patients", list_patients))
                app.add_handler(CommandHandler("update_token", update_token))
                app.add_handler(CommandHandler("register_bot", register_bot))
                app.add_handler(CommandHandler("list_bots", list_bots))
                app.add_handler(CommandHandler("update_prescription", update_prescription))

            # Lệnh cho bot riêng của bệnh nhân
            if is_private_bot and not is_doctor_bot:
                app.add_handler(CommandHandler("get_doctor_info", get_doctor_info))
                app.add_handler(CommandHandler("get_doctor_log", get_doctor_log_command))

            # Lệnh chỉ cho bot chính
            if not is_private_bot:
                app.add_handler(CommandHandler("start_bot", start_bot))
                app.add_handler(CommandHandler("stop_bot", stop_bot))
                app.add_handler(CommandHandler("delete_bot", delete_bot))
                app.add_handler(CommandHandler("start_all", start_all))

            app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
            app.add_error_handler(error_handler)

            await app.initialize()
            await app.start()
            await asyncio.sleep(1)
            await app.updater.start_polling(allowed_updates=Update.ALL_TYPES)
            bots[telegram_token] = {'application': app, 'bot': app.bot}
            logger.info(f"Bot {'riêng' if is_private_bot else 'chính'} với token {telegram_token} đã được khởi tạo")
            return app.bot
        except Exception as e:
            logger.error(f"Lỗi khi tạo bot với token {telegram_token}: {e}")
            raise ValueError(f"Lỗi khi tạo bot: {e}")

    # Kiểm tra trạng thái bot
    if not bots[telegram_token]['application'].updater.running:
        try:
            await bots[telegram_token]['application'].start()
            await bots[telegram_token]['application'].updater.start_polling(allowed_updates=Update.ALL_TYPES)
            logger.info(f"Bot với token {telegram_token} đã được khởi động lại")
        except Exception as e:
            logger.error(f"Lỗi khi khởi động lại bot {telegram_token}: {e}")
            raise ValueError(f"Lỗi khi khởi động lại bot: {e}")

    return bots[telegram_token]['bot']

async def check_permission(update: Update, context, require_doctor: bool = False, main_bot_only: bool = False, confirm_access: bool = False, patient_access: bool = False):
    telegram_id = str(update.effective_user.id)
    bot_token = context.bot.token

    # Bot chính có toàn quyền, không cần kiểm tra vai trò hoặc xác nhận
    if bot_token == DEFAULT_TELEGRAM_TOKEN:
        if main_bot_only:
            return True  # Lệnh chỉ dành cho bot chính (như /start_bot, /stop_bot) được phép
        return True  # Tất cả lệnh đều được phép trên bot chính

    # Kiểm tra cho bot riêng
    if main_bot_only:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bot chính mới có quyền thực hiện lệnh này!",
            parse_mode='Markdown'
        )
        return False

    user = get_user(telegram_id)
    if confirm_access:
        # Cho phép /confirm trên bot riêng nếu người dùng có token khớp
        if user and user['telegram_token'] == bot_token:
            return True
        # Cho phép /confirm nếu người dùng chưa có trong cơ sở dữ liệu
        conn = sqlite3.connect(DB_PATH)
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT user_id FROM users WHERE telegram_token = ?", (bot_token,))
            bot_user = cursor.fetchone()
            if bot_user:  # Bot riêng thuộc về một bác sĩ
                return True
            return False
        finally:
            conn.close()

    if not user:
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng xác nhận vai trò bằng `/confirm` trước!",
            parse_mode='Markdown'
        )
        return False

    if require_doctor and user['role'] != 'doctor':
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bác sĩ mới có quyền thực hiện lệnh này!",
            parse_mode='Markdown'
        )
        return False

    if patient_access and user['role'] != 'patient':
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Chỉ bệnh nhân mới có quyền thực hiện lệnh này!",
            parse_mode='Markdown'
        )
        return False

    return True

async def start(update: Update, context):
    logger.info(f"Nhận lệnh /start từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
            # Tự động xác nhận vai trò nếu đã có user
            context.args = [str(user['user_id']), telegram_id, user['role']]
            await confirm(update, context)
            return
        except Exception as e:
            logger.error(f"Lỗi khi khởi động bot riêng: {e}")
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"⚠️ Lỗi khởi động bot riêng: {str(e)}",
                parse_mode='Markdown'
            )

    if is_main_bot:
        welcome_message = (
            f"👋 *Chào bạn* (ID: `{telegram_id}`)!\n"
            "🌟 Đây là *bot chính*, bạn có toàn quyền thực hiện tất cả lệnh!\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò (tùy chọn)\n"
            "• `/add_doctor` - Thêm bác sĩ\n"
            "• `/add_patient` - Thêm bệnh nhân\n"
            "• `/list_patients` - Xem bệnh nhân\n"
            "• `/list_bots` - Xem danh sách bot\n"
            "• `/update_token` - Cập nhật token\n"
            "• `/register_bot` - Đăng ký bot\n"
            "• `/start_bot` - Khởi động bot\n"
            "• `/stop_bot` - Dừng bot\n"
            "• `/delete_bot` - Xóa bot\n"
            "• `/start_all` - Khởi động tất cả bot\n"
            "• `/update_prescription` - Cập nhật đơn thuốc\n"
            "• `/cancel` - Hủy thao tác\n"
        )
    else:
        welcome_message = (
            f"👋 *Chào bạn* (ID: `{telegram_id}`)!\n"
            "📌 Đây là bot riêng, bạn cần xác nhận vai trò bằng `/confirm`.\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò\n"
            "• `/cancel` - Hủy thao tác\n"
        )
        if user:
            if user['role'] == 'doctor':
                welcome_message = (
                    f"👋 *Chào bác sĩ {user['name']}* (ID: `{telegram_id}`)!\n"
                    "📋 *Lệnh khả dụng*:\n"
                    "• `/get_id` - Lấy Telegram ID\n"
                    "• `/confirm` - Xác nhận vai trò\n"
                    "• `/add_doctor` - Thêm bác sĩ\n"
                    "• `/add_patient` - Thêm bệnh nhân\n"
                    "• `/list_patients` - Xem bệnh nhân\n"
                    "• `/list_bots` - Xem danh sách bot\n"
                    "• `/update_token` - Cập nhật token\n"
                    "• `/register_bot` - Đăng ký bot\n"
                    "• `/update_prescription` - Cập nhật đơn thuốc\n"
                    "• `/cancel` - Hủy thao tác\n"
                )
            else:
                welcome_message = (
                    f"👋 *Chào bệnh nhân {user['name']}* (ID: `{telegram_id}`)!\n"
                    "📋 *Lệnh khả dụng*:\n"
                    "• `/get_id` - Lấy Telegram ID\n"
                    "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                    "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                    "• `/cancel` - Hủy thao tác\n"
                )

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=welcome_message.format(telegram_id=telegram_id),
        parse_mode='Markdown'
    )

# --- Sửa hàm get_id ---
async def get_id(update: Update, context):
    logger.info(f"Nhận lệnh /get_id từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    user = get_user(telegram_id)
    role = user['role'] if user else ("chính" if is_main_bot else "chưa xác nhận")
    name = user['name'] if user else "N/A"

    if is_main_bot:
        message = (
            f"📌 *Telegram ID*: `{telegram_id}`\n"
            f"👤 *Vai trò*: {role}\n"
            f"ℹ️ *Tên*: {name}\n"
            "🌟 Bạn đang dùng bot chính với toàn quyền!\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò (tùy chọn)\n"
            "• `/add_doctor` - Thêm bác sĩ\n"
            "• `/add_patient` - Thêm bệnh nhân\n"
            "• `/list_patients` - Xem bệnh nhân\n"
            "• `/list_bots` - Xem danh sách bot\n"
            "• `/update_token` - Cập nhật token\n"
            "• `/register_bot` - Đăng ký bot\n"
            "• `/start_bot` - Khởi động bot\n"
            "• `/stop_bot` - Dừng bot\n"
            "• `/delete_bot` - Xóa bot\n"
            "• `/start_all` - Khởi động tất cả bot\n"
            "• `/update_prescription` - Cập nhật đơn thuốc\n"
            "• `/cancel` - Hủy thao tác\n"
        )
    else:
        message = (
            f"📌 *Telegram ID*: `{telegram_id}`\n"
            f"👤 *Vai trò*: {role}\n"
            f"ℹ️ *Tên*: {name}\n"
            "📌 Vui lòng dùng `/confirm` để xác nhận vai trò.\n"
            "📋 *Lệnh khả dụng*:\n"
            "• `/get_id` - Lấy Telegram ID\n"
            "• `/confirm` - Xác nhận vai trò\n"
            "• `/cancel` - Hủy thao tác\n"
        )
        if user and user['role'] == 'doctor':
            message = (
                f"📌 *Telegram ID*: `{telegram_id}`\n"
                f"👤 *Vai trò*: {role}\n"
                f"ℹ️ *Tên*: {name}\n"
                "📋 *Lệnh khả dụng*:\n"
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/confirm` - Xác nhận vai trò\n"
                "• `/add_doctor` - Thêm bác sĩ\n"
                "• `/add_patient` - Thêm bệnh nhân\n"
                "• `/list_patients` - Xem bệnh nhân\n"
                "• `/list_bots` - Xem danh sách bot\n"
                "• `/update_token` - Cập nhật token\n"
                "• `/register_bot` - Đăng ký bot\n"
                "• `/update_prescription` - Cập nhật đơn thuốc\n"
                "• `/cancel` - Hủy thao tác\n"
            )
        elif user and user['role'] == 'patient':
            doctor = get_user_by_id(user['doctor_id']) if user['doctor_id'] else None
            doctor_info = (
                f"👨‍⚕️ *Bác sĩ phụ trách*: {doctor['name']} (ID: `{doctor['user_id']}`)\n"
                if doctor else
                f"❌ *Không tìm thấy bác sĩ phụ trách*\n"
            )
            message = (
                f"📌 *Telegram ID*: `{telegram_id}`\n"
                f"👤 *Vai trò*: {role}\n"
                f"ℹ️ *Tên*: {name}\n"
                f"{doctor_info}"
                "📋 *Lệnh khả dụng*:\n"
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                "• `/cancel` - Hủy thao tác\n"
            )

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def confirm(update: Update, context):
    logger.info(f"Nhận lệnh /confirm từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    is_main_bot = bot.token == DEFAULT_TELEGRAM_TOKEN

    # Kiểm tra quyền trên bot riêng
    if not is_main_bot and not await check_permission(update, context, confirm_access=True):
        return

    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
        except Exception as e:
            logger.error(f"Lỗi khi lấy bot riêng: {e}")
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"⚠️ Lỗi khi sử dụng bot riêng: {str(e)}",
                parse_mode='Markdown'
            )

    # Nếu người dùng đã xác nhận trên bot riêng
    if user and not is_main_bot and user['telegram_token'] == bot.token:
        message = (
            f"✅ *Xác nhận vai trò*: {user['role']} - {user['name']} (ID: `{telegram_id}`)\n"
            "📋 *Lệnh khả dụng*:\n"
        )
        if user['role'] == 'doctor':
            message += (
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/confirm` - Xác nhận vai trò\n"
                "• `/add_doctor` - Thêm bác sĩ\n"
                "• `/add_patient` - Thêm bệnh nhân\n"
                "• `/list_patients` - Xem bệnh nhân\n"
                "• `/list_bots` - Xem danh sách bot\n"
                "• `/update_token` - Cập nhật token\n"
                "• `/register_bot` - Đăng ký bot\n"
                "• `/update_prescription` - Cập nhật đơn thuốc\n"
                "• `/cancel` - Hủy thao tác\n"
            )
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=message,
                parse_mode='Markdown'
            )
            # Kiểm tra xem thông tin bác sĩ đã được cập nhật chưa
            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()
                cursor.execute("SELECT user_id FROM doctors_log WHERE user_id = ?", (user['user_id'],))
                if not cursor.fetchone():
                    context.user_data['expecting'] = 'doctor_log'
                    context.user_data['user_id'] = user['user_id']
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=(
                            "📝 *Cập nhật thông tin bác sĩ* (bắt buộc):\n"
                            "Vui lòng gửi thông tin theo mẫu:\n"
                            "```\n"
                            "Địa chỉ: <địa chỉ của bạn>\n"
                            "Số điện thoại: <số điện thoại>\n"
                            "Chuyên khoa: <chuyên khoa>\n"
                            "```\n"
                            "Ví dụ:\n"
                            "```\n"
                            "Địa chỉ: 123 Đường Láng, Hà Nội\n"
                            "Số điện thoại: 0987654321\n"
                            "Chuyên khoa: Nội khoa\n"
                            "```\n"
                            "❌ Gửi `/cancel` để hủy."
                        ),
                        parse_mode='Markdown'
                    )
            finally:
                conn.close()
        else:
            message += (
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                "• `/cancel` - Hủy thao tác\n"
            )
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=message,
                parse_mode='Markdown'
            )
        return

    # Xử lý xác nhận mới
    if len(context.args) != 3:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 *Xác nhận vai trò*\n"
                "Gửi lệnh theo định dạng:\n"
                "```\n/confirm <user_id> <telegram_id> <role>\n"
                "```\n"
                "Ví dụ: `/confirm 101 123456789 patient`\n"
                "• `<role>`: `doctor` hoặc `patient`\n"
                "❌ Gửi `/cancel` để hủy."
            ),
            parse_mode='Markdown'
        )
        return

    try:
        user_id = int(context.args[0])
        provided_telegram_id = str(context.args[1])
        role = context.args[2].lower()
        if role not in ['doctor', 'patient']:
            raise ValueError("Role phải là 'doctor' hoặc 'patient'")

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Kiểm tra telegram_id trùng lặp
        cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?", (provided_telegram_id, user_id))
        if cursor.fetchone():
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Telegram ID `{provided_telegram_id}` đã được liên kết với người dùng khác!",
                parse_mode='Markdown'
            )
            conn.close()
            return

        # Kiểm tra user_id tồn tại
        cursor.execute("SELECT user_id, name, role, doctor_id, telegram_token, bot_name FROM users WHERE user_id = ?", (user_id,))
        user_record = cursor.fetchone()

        if not user_record and role == 'doctor' and not is_main_bot:
            # Tạo bác sĩ mới trên bot riêng
            bot_name = f"Bot_{user_id}"
            cursor.execute("SELECT user_id FROM users WHERE user_id = ? OR bot_name = ?", (user_id, bot_name))
            if cursor.fetchone():
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=f"❌ ID `{user_id}` hoặc bot name `{bot_name}` đã tồn tại!",
                    parse_mode='Markdown'
                )
                conn.close()
                return

            cursor.execute("""
                INSERT INTO users (user_id, name, telegram_id, role, telegram_token, bot_name)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (user_id, f"Doctor_{user_id}", provided_telegram_id, 'doctor', bot.token, bot_name))
            conn.commit()
            export_to_csv()

            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=(
                    f"🎉 *Đã tạo và xác nhận bác sĩ mới*:\n"
                    f"• *ID*: `{user_id}`\n"
                    f"• *Tên*: Doctor_{user_id}\n"
                    f"• *Bot*: `{bot_name}`\n"
                    "🤖 Bot riêng của bạn đang chạy."
                ),
                parse_mode='Markdown'
            )
            context.user_data['expecting'] = 'doctor_log'
            context.user_data['user_id'] = user_id
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=(
                    "📝 *Cập nhật thông tin bác sĩ* (bắt buộc):\n"
                    "Vui lòng gửi thông tin theo mẫu:\n"
                    "```\n"
                    "Địa chỉ: <địa chỉ của bạn>\n"
                    "Số điện thoại: <số điện thoại>\n"
                    "Chuyên khoa: <chuyên khoa>\n"
                    "```\n"
                    "Ví dụ:\n"
                    "```\n"
                    "Địa chỉ: 123 Đường Láng, Hà Nội\n"
                    "Số điện thoại: 0987654321\n"
                    "Chuyên khoa: Nội khoa\n"
                    "```\n"
                    "❌ Gửi `/cancel` để hủy."
                ),
                parse_mode='Markdown'
            )
            conn.close()
            return

        if not user_record:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Không tìm thấy người dùng với ID: `{user_id}`",
                parse_mode='Markdown'
            )
            conn.close()
            return

        if user_record[2] != role:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Vai trò `{role}` không khớp với vai trò đã đăng ký (`{user_record[2]}`)!",
                parse_mode='Markdown'
            )
            conn.close()
            return

        # Cập nhật telegram_id
        cursor.execute("UPDATE users SET telegram_id = ? WHERE user_id = ?", (provided_telegram_id, user_id))
        conn.commit()
        export_to_csv()

        message = (
            f"✅ *Xác nhận vai trò*: {role} - {user_record[1]} (ID: `{user_id}`)\n"
            f"📌 *Telegram ID liên kết*: `{provided_telegram_id}`\n"
            "📋 *Lệnh khả dụng*:\n"
        )
        doctor_info = ""
        if role == 'patient' and user_record[3]:
            doctor = get_user_by_id(user_record[3])
            doctor_info = (
                f"👨‍⚕️ *Bác sĩ phụ trách*: {doctor['name']} (ID: `{doctor['user_id']}`)\n"
                if doctor else
                f"❌ *Không tìm thấy bác sĩ phụ trách*\n"
            )
            message += (
                f"{doctor_info}"
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                "• `/cancel` - Hủy thao tác\n"
            )
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=message,
                parse_mode='Markdown'
            )

            # Gửi hướng dẫn cho bệnh nhân qua bot riêng
            if user_record[4] and user_record[4] != DEFAULT_TELEGRAM_TOKEN:
                try:
                    patient_bot = await get_bot(user_record[4], is_private_bot=True, is_doctor_bot=False)
                    await patient_bot.send_message(
                        chat_id=provided_telegram_id,
                        text=(
                            f"👋 *Chào bệnh nhân {user_record[1]}* (ID: `{user_id}`)!\n"
                            f"{doctor_info}"
                            "📋 *Lệnh khả dụng*:\n"
                            "• `/get_id` - Lấy Telegram ID\n"
                            "• `/get_doctor_info` - Xem thông tin bác sĩ\n"
                            "• `/get_doctor_log` - Xem chi tiết bác sĩ\n"
                            "• `/cancel` - Hủy thao tác\n"
                            "🚀 Bot riêng của bạn sẽ gửi nhắc nhở uống thuốc và tái khám!"
                        ),
                        parse_mode='Markdown'
                    )
                    logger.info(f"Đã gửi hướng dẫn lệnh tới bệnh nhân {user_record[1]} qua bot riêng")
                except Exception as e:
                    logger.error(f"Lỗi khi gửi hướng dẫn tới bệnh nhân {user_record[1]}: {e}")
        else:
            message += (
                "• `/get_id` - Lấy Telegram ID\n"
                "• `/confirm` - Xác nhận vai trò\n"
                "• `/add_doctor` - Thêm bác sĩ\n"
                "• `/add_patient` - Thêm bệnh nhân\n"
                "• `/list_patients` - Xem bệnh nhân\n"
                "• `/list_bots` - Xem danh sách bot\n"
                "• `/update_token` - Cập nhật token\n"
                "• `/register_bot` - Đăng ký bot\n"
                "• `/update_prescription` - Cập nhật đơn thuốc\n"
                "• `/cancel` - Hủy thao tác\n"
            )
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=message,
                parse_mode='Markdown'
            )
            # Yêu cầu cập nhật thông tin bác sĩ nếu chưa có
            cursor.execute("SELECT user_id FROM doctors_log WHERE user_id = ?", (user_id,))
            if not cursor.fetchone():
                context.user_data['expecting'] = 'doctor_log'
                context.user_data['user_id'] = user_id
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        "📝 *Cập nhật thông tin bác sĩ* (bắt buộc):\n"
                        "Vui lòng gửi thông tin theo mẫu:\n"
                        "```\n"
                        "Địa chỉ: <địa chỉ của bạn>\n"
                        "Số điện thoại: <số điện thoại>\n"
                        "Chuyên khoa: <chuyên khoa>\n"
                        "```\n"
                        "Ví dụ:\n"
                        "```\n"
                        "Địa chỉ: 123 Đường Láng, Hà Nội\n"
                        "Số điện thoại: 0987654321\n"
                        "Chuyên khoa: Nội khoa\n"
                        "```\n"
                        "❌ Gửi `/cancel` để hủy."
                    ),
                    parse_mode='Markdown'
                )

        conn.close()
    except ValueError as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi: {str(e)}",
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Lỗi khi xử lý /confirm: {e}")
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi không xác định: {str(e)}",
            parse_mode='Markdown'
        )
        if 'conn' in locals():
            conn.close()

async def add_doctor(update: Update, context):
    logger.info(f"Nhận lệnh /add_doctor từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'doctor'
    await bot.send_message(
    chat_id=update.effective_chat.id,
    text=(
        "📝 *Thêm bác sĩ mới*\n"
        "Gửi thông tin theo mẫu JSON sau:\n"
        "```json\n"
        "{\n"
        "  \"user_id\": <số nguyên, ví dụ: 2>,\n"
        "  \"name\": \"<tên bác sĩ, ví dụ: Dr. Hoa>\",\n"
        "  \"role\": \"doctor\",\n"
        "  \"telegram_token\": \"<token bot riêng, tùy chọn>\",\n"
        "  \"bot_name\": \"<tên bot, tùy chọn, ví dụ: Bot_2>\"\n"
        "}\n"
        "```\n"
        "📌 *Lưu ý*:\n"
        "• `telegram_token` và `bot_name` là tùy chọn.\n"
        "• Nếu không cung cấp `bot_name`, hệ thống sẽ gán `Bot_<user_id>`.\n"
        "• Nếu cung cấp `telegram_token`, bot riêng sẽ tự động chạy.\n"
        "• Bác sĩ cần dùng `/confirm` để liên kết Telegram ID.\n"
        "❌ Gửi `/cancel` để hủy."
    ),
    parse_mode='Markdown'
)

async def add_patient(update: Update, context):
    logger.info(f"Nhận lệnh /add_patient từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'patient'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 *Thêm bệnh nhân mới*\n"
            "Gửi thông tin theo mẫu JSON sau:\n"
            "```json\n"
            "{\n"
            "  \"user_id\": <số nguyên, ví dụ: 101>,\n"
            "  \"name\": \"<tên bệnh nhân, ví dụ: Nguyễn Văn A>\",\n"
            "  \"role\": \"patient\",\n"
            "  \"doctor_id\": <ID bác sĩ, ví dụ: 1>,\n"
            "  \"telegram_token\": \"<token bot riêng, tùy chọn>\",\n"
            "  \"information\": {\n"
            "    \"tuoi\": <tuổi, ví dụ: 65>,\n"
            "    \"benh\": \"<bệnh, ví dụ: Tiểu đường>\",\n"
            "    \"thuoc\": \"<thuốc, ví dụ: Metformin>\",\n"
            "    \"sang\": \"<giờ sáng, ví dụ: 07:30, tùy chọn>\",\n"
            "    \"trua\": \"<giờ trưa, ví dụ: 12:00, tùy chọn>\",\n"
            "    \"toi\": \"<giờ tối, ví dụ: 18:00, tùy chọn>\",\n"
            "    \"tai_kham\": \"<ngày tái khám, ví dụ: 2025-06-01, tùy chọn>\"\n"
            "  }\n"
            "}\n"
            "```\n"
            "📌 *Lưu ý*:\n"
            "• `telegram_token`, `information`, `sang`, `trua`, `toi`, `tai_kham` là tùy chọn.\n"
            "• Nếu có `tai_kham`, bot sẽ nhắc tái khám trước 2 ngày, 1 ngày và ngày tái khám.\n"
            "• Cần ít nhất một trong `sang`, `trua`, `toi` để nhắc uống thuốc.\n"
            "• Bệnh nhân cần dùng `/cofirm` để liên kết Telegram ID.\n"
            "❌ Gửi `/cancel` để hủy."
        ),
        parse_mode='Markdown'
    )

async def update_token(update: Update, context):
    logger.info(f"Nhận lệnh /update_token từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    context.user_data['expecting'] = 'token'
    context.user_data['user_id'] = user['user_id'] if user else None
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 *Cập nhật token bot riêng*\n"
            "Vui lòng gửi Telegram token theo mẫu:\n"
            "```\n"
            "<telegram_token>\n"
            "```\n"
            "Ví dụ: `1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p`\n"
            "❌ Gửi `/cancel` để hủy."
        ),
        parse_mode='Markdown'
    )

async def update_prescription(update: Update, context):
    logger.info(f"Nhận lệnh /update_prescription từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                "📝 *Cập nhật đơn thuốc*\n"
                "Gửi lệnh theo định dạng:\n"
                "```\n/update_prescription <patient_id>\n"
                "```\n"
                "Ví dụ: `/update_prescription 101`\n"
                "❌ Gửi `/cancel` để hủy."
            ),
            parse_mode='Markdown'
        )
        return

    try:
        patient_id = int(context.args[0])
        patient = get_user_by_id(patient_id)
        if not patient or patient['role'] != 'patient' or patient['doctor_id'] != user['user_id']:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Không tìm thấy bệnh nhân với ID `{patient_id}` hoặc bạn không phụ trách bệnh nhân này!",
                parse_mode='Markdown'
            )
            return

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT tai_kham FROM information WHERE user_id = ?", (patient_id,))
        tai_kham = cursor.fetchone()
        conn.close()

        current_date = datetime.now(timezone(timedelta(hours=7))).date()
        if not tai_kham or not tai_kham[0] or datetime.strptime(tai_kham[0], "%Y-%m-%d").date() > current_date:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Chỉ có thể cập nhật đơn thuốc sau ngày tái khám `{tai_kham[0]}`!",
                parse_mode='Markdown'
            )
            return

        context.user_data['expecting'] = 'prescription'
        context.user_data['patient_id'] = patient_id
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                f"📝 *Cập nhật đơn thuốc cho bệnh nhân {patient['name']}* (ID: `{patient_id}`)\n"
                "Gửi thông tin theo mẫu JSON sau:\n"
                "```json\n"
                "{\n"
                "  \"benh\": \"<bệnh mới, ví dụ: Tiểu đường>\",\n"
                "  \"thuoc\": \"<thuốc mới, ví dụ: Insulin>\",\n"
                "  \"sang\": \"<giờ sáng, ví dụ: 07:30, tùy chọn>\",\n"
                "  \"trua\": \"<giờ trưa, ví dụ: 12:00, tùy chọn>\",\n"
                "  \"toi\": \"<giờ tối, ví dụ: 18:00, tùy chọn>\",\n"
                "  \"tai_kham\": \"<ngày tái khám mới, ví dụ: 2025-07-01, tùy chọn>\"\n"
                "}\n"
                "```\n"
                "📌 *Lưu ý*:\n"
                "• Cần ít nhất `benh` và `thuoc`.\n"
                "• `sang`, `trua`, `toi`, `tai_kham` là tùy chọn.\n"
                "❌ Gửi `/cancel` để hủy."
            ),
            parse_mode='Markdown'
        )
    except ValueError:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp `patient_id` là số nguyên!",
            parse_mode='Markdown'
        )

async def get_doctor_info(update: Update, context):
    logger.info(f"Nhận lệnh /get_doctor_info từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, patient_access=True):
        return

    telegram_id = str(update.effective_user.id)
    bot = context.bot
    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    doctor = get_user_by_id(user['doctor_id']) if user['doctor_id'] else None
    if not doctor:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Không tìm thấy thông tin bác sĩ phụ trách.",
            parse_mode='Markdown'
        )
        return

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "👨‍⚕️ *Bác sĩ phụ trách*:\n"
            f"• *ID*: `{doctor['user_id']}`\n"
            f"• *Tên*: {doctor['name']}\n"
            f"• *Bot*: {doctor['bot_name'] or 'Không có'}"
        ),
        parse_mode='Markdown'
    )

async def get_doctor_log_command(update: Update, context):
    logger.info(f"Nhận lệnh /get_doctor_log từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, patient_access=True):
        return

    telegram_id = str(update.effective_user.id)
    bot = context.bot
    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    doctor_log = get_doctor_log(user['doctor_id']) if user['doctor_id'] else None
    if not doctor_log:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Không tìm thấy thông tin chi tiết bác sĩ phụ trách.",
            parse_mode='Markdown'
        )
        return

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "👨‍⚕️ *Thông tin bác sĩ phụ trách*:\n"
            f"• *ID*: `{doctor_log['user_id']}`\n"
            f"• *Tên*: {doctor_log['name']}\n"
            f"• *Địa chỉ*: {doctor_log['address'] or 'N/A'}\n"
            f"• *Số điện thoại*: {doctor_log['phone'] or 'N/A'}\n"
            f"• *Chuyên khoa*: {doctor_log['specialty'] or 'N/A'}\n"
            f"• *Cập nhật lúc*: {doctor_log['updated_at'] or 'N/A'}"
        ),
        parse_mode='Markdown'
    )

async def list_patients(update: Update, context):
    logger.info(f"Nhận lệnh /list_patients từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    patients = get_patients(user['user_id'] if user else 1)
    if not patients:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Bạn chưa có bệnh nhân nào.",
            parse_mode='Markdown'
        )
        return

    message = f"📋 *Danh sách bệnh nhân của bác sĩ {user['name']}* (ID: `{user['user_id']}`):\n\n"
    for p in patients:
        message += (
            f"🧑 *Bệnh nhân {p['name']}* (ID: `{p['user_id']}`):\n"
            f"• *Tuổi*: {p['tuoi'] or 'N/A'}\n"
            f"• *Bệnh*: {p['benh'] or 'N/A'}\n"
            f"• *Thuốc*: {p['thuoc'] or 'N/A'}\n"
            f"• *Nhắc nhở uống thuốc*:\n"
            f"  - Sáng: {p['sang'] or 'N/A'}\n"
            f"  - Trưa: {p['trua'] or 'N/A'}\n"
            f"  - Tối: {p['toi'] or 'N/A'}\n"
            f"• *Tái khám*: {p['tai_kham'] or 'N/A'}\n"
            f"• *Telegram ID*: {p['telegram_id'] or 'Chưa xác nhận'}\n"
            f"• *Bot riêng*: {'Có' if p['telegram_token'] else 'Không'}\n"
            f"---\n"
        )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def list_bots(update: Update, context):
    logger.info(f"Nhận lệnh /list_bots từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_id, telegram_token, name, bot_name, role FROM users WHERE telegram_token IS NOT NULL")
    bot_mappings = cursor.fetchall()
    conn.close()

    if not bot_mappings:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Chưa có bot riêng nào được thiết lập.",
            parse_mode='Markdown'
        )
        return

    message = "🤖 *Danh sách bot riêng*:\n\n"
    for mapping in bot_mappings:
        telegram_id, telegram_token, name, bot_name, role = mapping
        if telegram_token and telegram_token.lower() != 'nan':
            status = "✅ Đang chạy" if telegram_token in bots and bots[telegram_token]['application'].updater.running else "❌ Không chạy"
            message += (
                f"{'👨‍⚕️' if role == 'doctor' else '🧑'} *{role.capitalize()}*: {name}\n"
                f"🆔 *Telegram ID*: {telegram_id or 'Chưa xác nhận'}\n"
                f"🤖 *Bot*: {bot_name}\n"
                f"📊 *Trạng thái*: {status}\n"
                f"---\n"
            )
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def start_bot(update: Update, context):
    logger.info(f"Nhận lệnh /start_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp `bot_name`. Ví dụ: `/start_bot Bot_5`",
            parse_mode='Markdown'
        )
        return

    bot_name = context.args[0]
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token, role FROM users WHERE bot_name = ?", (bot_name,))
    result = cursor.fetchone()
    conn.close()

    if not result:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Không tìm thấy bot với `bot_name`: `{bot_name}`",
            parse_mode='Markdown'
        )
        return

    telegram_token, role = result
    if not telegram_token or telegram_token.lower() == 'nan':
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Bot `{bot_name}` không có token hợp lệ. Cập nhật bằng `/update_token`.",
            parse_mode='Markdown'
        )
        return

    if telegram_token in bots and bots[telegram_token]['application'].updater.running:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"ℹ️ Bot `{bot_name}` đã đang chạy!",
            parse_mode='Markdown'
        )
        return

    try:
        await get_bot(telegram_token, is_private_bot=True, is_doctor_bot=role == 'doctor')
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"✅ Đã khởi động bot: `{bot_name}`",
            parse_mode='Markdown'
        )
    except Exception as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi khi khởi động bot `{bot_name}`: {str(e)}",
            parse_mode='Markdown'
        )

async def start_all(update: Update, context):
    logger.info(f"Nhận lệnh /start_all từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token, bot_name, role FROM users WHERE telegram_token IS NOT NULL AND telegram_token != ?", (DEFAULT_TELEGRAM_TOKEN,))
    bots_to_start = cursor.fetchall()
    conn.close()

    if not bots_to_start:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Không có bot riêng nào để khởi động.",
            parse_mode='Markdown'
        )
        return

    started = []
    failed = []
    for telegram_token, bot_name, role in bots_to_start:
        if telegram_token and telegram_token.lower() != 'nan':
            if telegram_token in bots and bots[telegram_token]['application'].updater.running:
                continue
            try:
                await get_bot(telegram_token, is_private_bot=True, is_doctor_bot=role == 'doctor')
                started.append(bot_name)
            except Exception as e:
                failed.append((bot_name, str(e)))

    message = "🤖 *Kết quả khởi động tất cả bot*:\n\n"
    if started:
        message += "✅ *Đã khởi động*:\n" + "\n".join(f"• `{name}`" for name in started) + "\n"
    if failed:
        message += "❌ *Thất bại*:\n" + "\n".join(f"• `{name}`: {error}" for name, error in failed)
    if not started and not failed:
        message += "ℹ️ Tất cả bot đã đang chạy!"

    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=message,
        parse_mode='Markdown'
    )

async def stop_bot(update: Update, context):
    logger.info(f"Nhận lệnh /stop_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp `bot_name`. Ví dụ: `/stop_bot Bot_5`",
            parse_mode='Markdown'
        )
        return

    bot_name = context.args[0]
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token FROM users WHERE bot_name = ?", (bot_name,))
    result = cursor.fetchone()
    conn.close()

    if not result:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Không tìm thấy bot với `bot_name`: `{bot_name}`",
            parse_mode='Markdown'
        )
        return

    telegram_token = result[0]
    if telegram_token in bots:
        try:
            await bots[telegram_token]['application'].updater.stop()
            await bots[telegram_token]['application'].stop()
            await bots[telegram_token]['application'].shutdown()
            del bots[telegram_token]
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"✅ Đã dừng bot: `{bot_name}`",
                parse_mode='Markdown'
            )
        except Exception as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi khi dừng bot `{bot_name}`: {str(e)}",
                parse_mode='Markdown'
            )
    else:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"ℹ️ Bot `{bot_name}` không đang chạy.",
            parse_mode='Markdown'
        )

async def delete_bot(update: Update, context):
    logger.info(f"Nhận lệnh /delete_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, main_bot_only=True):
        return

    bot = context.bot
    if not context.args:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="❌ Vui lòng cung cấp `bot_name`. Ví dụ: `/delete_bot Bot_5`",
            parse_mode='Markdown'
        )
        return

    bot_name = context.args[0]
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT telegram_token, user_id, role FROM users WHERE bot_name = ?", (bot_name,))
    result = cursor.fetchone()

    if not result:
        conn.close()
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Không tìm thấy bot với `bot_name`: `{bot_name}`",
            parse_mode='Markdown'
        )
        return

    telegram_token, user_id, role = result
    try:
        if telegram_token and telegram_token in bots:
            await bots[telegram_token]['application'].updater.stop()
            await bots[telegram_token]['application'].stop()
            await bots[telegram_token]['application'].shutdown()
            del bots[telegram_token]

        if role == 'patient':
            cursor.execute("DELETE FROM information WHERE user_id = ?", (user_id,))
        cursor.execute("DELETE FROM users WHERE user_id = ?", (user_id,))
        cursor.execute("DELETE FROM doctors_log WHERE user_id = ?", (user_id,))
        conn.commit()
        export_to_csv()

        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"✅ Đã xóa bot: `{bot_name}` và thông tin {role} (ID: `{user_id}`).",
            parse_mode='Markdown'
        )
    except Exception as e:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text=f"❌ Lỗi khi xóa bot `{bot_name}`: {str(e)}",
            parse_mode='Markdown'
        )
    finally:
        conn.close()

async def cancel(update: Update, context):
    logger.info(f"Nhận lệnh /cancel từ Telegram ID: {update.effective_user.id}")
    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')

    context.user_data.pop('expecting', None)
    context.user_data.pop('user_id', None)
    context.user_data.pop('patient_id', None)
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text="❌ Đã hủy thao tác.",
        parse_mode='Markdown'
    )

async def register_bot(update: Update, context):
    logger.info(f"Nhận lệnh /register_bot từ Telegram ID: {update.effective_user.id}")
    if not await check_permission(update, context, require_doctor=True):
        return

    bot = context.bot
    user = get_user(str(update.effective_user.id))
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=True)

    context.user_data['expecting'] = 'register_bot'
    await bot.send_message(
        chat_id=update.effective_chat.id,
        text=(
            "📝 *Đăng ký bot riêng*\n"
            "Vui lòng gửi thông tin theo mẫu:\n"
            "```\n"
            "Tên: <tên bác sĩ>\n"
            "Token: <telegram_token>\n"
            "Bot Name: <tên bot, tùy chọn>\n"
            "```\n"
            "Ví dụ:\n"
            "```\n"
            "Tên: Dr. Minh\n"
            "Token: 1234567890:AAF1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p\n"
            "Bot Name: Bot_3\n"
            "```\n"
            "📌 *Lưu ý*:\n"
            "• Nếu không cung cấp `Bot Name`, hệ thống sẽ gán `Bot_<user_id>`.\n"
            "• Token phải hợp lệ từ @BotFather.\n"
            "❌ Gửi `/cancel` để hủy."
        ),
        parse_mode='Markdown'
    )

async def send_reminders():
    logger.info("Kiểm tra lịch nhắc nhở")
    try:
        current_time = datetime.now(timezone(timedelta(hours=7))).strftime("%H:%M")
        current_date = datetime.now(timezone(timedelta(hours=7))).date()
        reminder_dates = [
            (current_date + timedelta(days=2)).strftime("%Y-%m-%d"),
            (current_date + timedelta(days=1)).strftime("%Y-%m-%d"),
            current_date.strftime("%Y-%m-%d")
        ]

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT u.user_id, u.name, u.telegram_id, u.telegram_token, u.doctor_id, i.thuoc, i.sang, i.trua, i.toi, i.tai_kham
            FROM users u
            JOIN information i ON u.user_id = i.user_id
            WHERE u.role = 'patient' AND (
                i.sang IS NOT NULL OR i.trua IS NOT NULL OR i.toi IS NOT NULL OR i.tai_kham IN (?, ?, ?)
            )
        """, reminder_dates)
        patients = cursor.fetchall()
        conn.close()

        for patient in patients:
            user_id, name, telegram_id, telegram_token, doctor_id, thuoc, sang, trua, toi, tai_kham = patient
            try:
                target_bot = None
                if telegram_token and telegram_token.lower() != 'nan':
                    target_bot = await get_bot(telegram_token, is_private_bot=True, is_doctor_bot=False)
                else:
                    doctor = get_user_by_id(doctor_id)
                    if doctor and doctor['telegram_token'] and doctor['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
                        target_bot = await get_bot(doctor['telegram_token'], is_private_bot=True, is_doctor_bot=True)
                    else:
                        target_bot = await get_bot(DEFAULT_TELEGRAM_TOKEN)

                if telegram_id:
                    # Nhắc uống thuốc
                    for time_slot, label in [(sang, "Sáng"), (trua, "Trưa"), (toi, "Tối")]:
                        if time_slot and is_time_match(time_slot, current_time):
                            await target_bot.send_message(
                                chat_id=telegram_id,
                                text=(
                                    f"⏰ *Nhắc nhở uống thuốc ({label})*:\n"
                                    f"• *Bệnh nhân*: {name}\n"
                                    f"• *Thuốc*: {thuoc}\n"
                                    f"• *Thời gian*: {time_slot}"
                                ),
                                parse_mode='Markdown'
                            )
                            logger.info(f"Đã gửi nhắc nhở uống thuốc ({label}) cho bệnh nhân {name} (ID: {user_id})")

                    # Nhắc tái khám
                    if tai_kham and tai_kham in reminder_dates:
                        day_label = "hôm nay" if tai_kham == current_date.strftime("%Y-%m-%d") else (
                            "ngày mai" if tai_kham == (current_date + timedelta(days=1)).strftime("%Y-%m-%d") else "sau 2 ngày"
                        )
                        await target_bot.send_message(
                            chat_id=telegram_id,
                            text=(
                                f"🩺 *Nhắc nhở tái khám* ({day_label}):\n"
                                f"• *Bệnh nhân*: {name}\n"
                                f"• *Ngày tái khám*: {tai_kham}\n"
                                f"• Vui lòng liên hệ bác sĩ để đặt lịch!"
                            ),
                            parse_mode='Markdown'
                        )
                        logger.info(f"Đã gửi nhắc nhở tái khám ({day_label}) cho bệnh nhân {name} (ID: {user_id})")
            except Exception as e:
                logger.error(f"Lỗi khi gửi nhắc nhở cho bệnh nhân {name} (ID: {user_id}): {e}")
    except Exception as e:
        logger.error(f"Lỗi khi kiểm tra lịch nhắc nhở: {e}")

def schedule_reminders():
    loop = asyncio.get_event_loop()
    async def periodic():
        while True:
            await send_reminders()
            await asyncio.sleep(60)
    loop.create_task(periodic())

async def handle_message(update: Update, context):
    logger.info(f"Nhận tin nhắn từ Telegram ID: {update.effective_user.id}")
    telegram_id = str(update.effective_user.id)
    bot = context.bot
    user = get_user(telegram_id)
    if user and user['telegram_token'] and user['telegram_token'] != DEFAULT_TELEGRAM_TOKEN:
        try:
            bot = await get_bot(user['telegram_token'], is_private_bot=True, is_doctor_bot=user['role'] == 'doctor')
        except Exception as e:
            logger.error(f"Lỗi khi lấy bot riêng cho Telegram ID {telegram_id}: {str(e)}")
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi khi sử dụng bot riêng: {str(e)}",
                parse_mode='Markdown'
            )
            return

    expecting = context.user_data.get('expecting')
    if not expecting:
        await bot.send_message(
            chat_id=update.effective_chat.id,
            text="📋 Vui lòng dùng `/add_doctor`, `/add_patient`, `/update_token`, `/register_bot`, hoặc `/update_prescription` trước!",
            parse_mode='Markdown'
        )
        return

    if expecting == 'token':
        token = update.message.text.strip()
        user_id = context.user_data.get('user_id')
        try:
            is_doctor_token = user and user['role'] == 'doctor'
            bot_instance = await get_bot(token, is_private_bot=True, is_doctor_bot=is_doctor_token)
            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()
                if not user_id:
                    cursor.execute("SELECT MAX(user_id) FROM users")
                    max_id = cursor.fetchone()[0] or 0
                    new_id = max_id + 1
                    bot_name = f"Bot_{new_id}"
                    cursor.execute("""
                        INSERT INTO users (user_id, name, telegram_id, role, telegram_token, bot_name)
                        VALUES (?, ?, ?, ?, ?, ?)
                    """, (new_id, f"Doctor_{new_id}", telegram_id, 'doctor', token, bot_name))
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=(
                            f"🎉 *Đã tạo bác sĩ mới*:\n"
                            f"• *ID*: `{new_id}`\n"
                            f"• *Tên*: Doctor_{new_id}\n"
                            f"• *Bot*: `{bot_name}`\n"
                            f"🤖 Bot riêng của bạn đang chạy."
                        ),
                        parse_mode='Markdown'
                    )
                else:
                    cursor.execute("SELECT role FROM users WHERE user_id = ?", (user_id,))
                    role = cursor.fetchone()
                    if not role:
                        raise ValueError(f"Không tìm thấy người dùng với ID `{user_id}`")
                    cursor.execute("UPDATE users SET telegram_token = ? WHERE user_id = ?", (token, user_id))
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=(
                            f"🎉 *Đã cập nhật token* cho {'bác sĩ' if role[0] == 'doctor' else 'bệnh nhân'} (ID: `{user_id}`).\n"
                            f"🤖 Bot riêng của bạn đang chạy."
                        ),
                        parse_mode='Markdown'
                    )
                conn.commit()
                export_to_csv()
            finally:
                conn.close()
            context.user_data.pop('expecting', None)
            context.user_data.pop('user_id', None)
        except Exception as e:
            logger.error(f"Lỗi khi xử lý token từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}\nℹ️ Kiểm tra token hoặc tạo mới qua @BotFather.",
                parse_mode='Markdown'
            )
        return

    if expecting == 'register_bot':
        try:
            if not user or user['role'] != 'doctor' or not user['telegram_id']:
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text="❌ Chỉ bác sĩ đã xác nhận vai trò (`/cofirm`) mới có thể đăng ký bot riêng!",
                    parse_mode='Markdown'
                )
                return

            lines = update.message.text.strip().split('\n')
            data = {'role': 'doctor'}
            for line in lines:
                if line.startswith('Tên:'):
                    data['name'] = line.replace('Tên:', '').strip()
                elif line.startswith('Token:'):
                    data['telegram_token'] = line.replace('Token:', '').strip()
                elif line.startswith('Bot Name:'):
                    data['bot_name'] = line.replace('Bot Name:', '').strip()

            if not all(key in data for key in ['name', 'telegram_token']):
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text="❌ Thiếu `Tên` hoặc `Token`. Vui lòng gửi đúng định dạng:\n```\nTên: <tên>\nToken: <token>\nBot Name: <tên bot, tùy chọn>\n```",
                    parse_mode='Markdown'
                )
                return

            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()

                # Kiểm tra token trùng lặp
                cursor.execute("SELECT user_id FROM users WHERE telegram_token = ?", (data['telegram_token'],))
                if cursor.fetchone():
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text="❌ Token đã được sử dụng bởi người dùng khác!",
                        parse_mode='Markdown'
                    )
                    return

                # Tạo user_id mới
                cursor.execute("SELECT MAX(user_id) FROM users")
                max_id = cursor.fetchone()[0] or 0
                data['user_id'] = max_id + 1
                data['bot_name'] = data.get('bot_name', f"Bot_{data['user_id']}")

                # Kiểm tra bot_name trùng lặp
                cursor.execute("SELECT user_id FROM users WHERE bot_name = ?", (data['bot_name'],))
                if cursor.fetchone():
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ Tên bot `{data['bot_name']}` đã tồn tại! Vui lòng chọn tên khác.",
                        parse_mode='Markdown'
                    )
                    return

                data['telegram_id'] = telegram_id
                await insert_to_db('users', data)
                try:
                    await get_bot(data['telegram_token'], is_private_bot=True, is_doctor_bot=True)
                except Exception as e:
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"⚠️ Đã thêm bác sĩ nhưng lỗi khi khởi động bot riêng: {str(e)}",
                        parse_mode='Markdown'
                    )
                    context.user_data.pop('expecting', None)
                    context.user_data.pop('user_id', None)
                    return

                conn.commit()
                export_to_csv()

                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 *Đã đăng ký bot riêng*:\n"
                        f"• *ID*: `{data['user_id']}`\n"
                        f"• *Tên*: {data['name']}\n"
                        f"• *Bot*: `{data['bot_name']}`\n"
                        f"🤖 Bot riêng đang chạy.\n"
                        f"📌 Bác sĩ đã được liên kết với Telegram ID `{telegram_id}`."
                    ),
                    parse_mode='Markdown'
                )
                context.user_data.pop('expecting', None)
                context.user_data.pop('user_id', None)
            finally:
                conn.close()
        except Exception as e:
            logger.error(f"Lỗi khi xử lý /register_bot từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}\nℹ️ Vui lòng kiểm tra lại dữ liệu hoặc liên hệ hỗ trợ.",
                parse_mode='Markdown'
            )
        return

    if expecting == 'doctor':
        try:
            data = json.loads(update.message.text.strip())
            if not isinstance(data, dict):
                raise ValueError("Dữ liệu phải là JSON object")

            required_fields = ['user_id', 'name', 'role']
            if not all(field in data for field in required_fields) or data['role'] != 'doctor':
                raise ValueError("Thiếu các trường bắt buộc (`user_id`, `name`, `role`) hoặc `role` không phải `doctor`")

            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()

                # Kiểm tra user_id và bot_name trùng lặp
                bot_name = data.get('bot_name', f"Bot_{data['user_id']}")
                cursor.execute("SELECT user_id FROM users WHERE user_id = ? OR bot_name = ?",
                              (data['user_id'], bot_name))
                existing = cursor.fetchone()
                if existing:
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ ID `{data['user_id']}` hoặc bot name `{bot_name}` đã tồn tại!",
                        parse_mode='Markdown'
                    )
                    return

                # Kiểm tra telegram_id (nếu có)
                if data.get('telegram_id'):
                    cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?",
                                  (data['telegram_id'], data['user_id']))
                    if cursor.fetchone():
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"❌ Telegram ID `{data['telegram_id']}` đã được liên kết với người dùng khác!",
                            parse_mode='Markdown'
                        )
                        return

                data['bot_name'] = bot_name
                await insert_to_db('users', data)
                if data.get('telegram_token') and data['telegram_token'].lower() != 'nan':
                    try:
                        await get_bot(data['telegram_token'], is_private_bot=True, is_doctor_bot=True)
                    except Exception as e:
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"⚠️ Đã thêm bác sĩ nhưng lỗi khi khởi động bot riêng: {str(e)}",
                            parse_mode='Markdown'
                        )
                        context.user_data.pop('expecting', None)
                        return

                conn.commit()
                export_to_csv()

                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 *Đã thêm bác sĩ*:\n"
                        f"• *ID*: `{data['user_id']}`\n"
                        f"• *Tên*: {data['name']}\n"
                        f"• *Bot*: `{data['bot_name']}`\n"
                        f"📌 Bác sĩ cần dùng `/cofirm {data['user_id']} <telegram_id> doctor` để xác nhận vai trò."
                    ),
                    parse_mode='Markdown'
                )
                context.user_data.pop('expecting', None)
            finally:
                conn.close()
        except json.JSONDecodeError:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Dữ liệu không phải JSON hợp lệ. Vui lòng gửi đúng định dạng JSON!",
                parse_mode='Markdown'
            )
        except ValueError as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}",
                parse_mode='Markdown'
            )
        except Exception as e:
            logger.error(f"Lỗi khi xử lý /add_doctor từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi không xác định: {str(e)}\nℹ️ Vui lòng kiểm tra lại dữ liệu hoặc liên hệ hỗ trợ.",
                parse_mode='Markdown'
            )
        return

    if expecting == 'patient':
        try:
            data = json.loads(update.message.text.strip())
            if not isinstance(data, dict):
                raise ValueError("Dữ liệu phải là JSON object")

            required_fields = ['user_id', 'name', 'role', 'doctor_id']
            if not all(field in data for field in required_fields) or data['role'] != 'patient':
                raise ValueError("Thiếu các trường bắt buộc (`user_id`, `name`, `role`, `doctor_id`) hoặc `role` không phải `patient`")

            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()

                # Kiểm tra bác sĩ tồn tại
                cursor.execute("SELECT user_id, role, telegram_id, telegram_token FROM users WHERE user_id = ?",
                              (data['doctor_id'],))
                doctor = cursor.fetchone()
                if not doctor or doctor[1] != 'doctor':
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ Bác sĩ với ID `{data['doctor_id']}` không tồn tại hoặc không phải bác sĩ!",
                        parse_mode='Markdown'
                    )
                    return

                # Kiểm tra user_id và bot_name trùng lặp
                bot_name = data.get('bot_name', f"Bot_{data['user_id']}")
                cursor.execute("SELECT user_id FROM users WHERE user_id = ? OR bot_name = ?",
                              (data['user_id'], bot_name))
                existing = cursor.fetchone()
                if existing:
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ ID `{data['user_id']}` hoặc bot name `{bot_name}` đã tồn tại!",
                        parse_mode='Markdown'
                    )
                    return

                # Kiểm tra telegram_id (nếu có)
                if data.get('telegram_id'):
                    cursor.execute("SELECT user_id FROM users WHERE telegram_id = ? AND user_id != ?",
                                  (data['telegram_id'], data['user_id']))
                    if cursor.fetchone():
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"❌ Telegram ID `{data['telegram_id']}` đã được liên kết với người dùng khác!",
                            parse_mode='Markdown'
                        )
                        return

                data['bot_name'] = bot_name
                await insert_to_db('users', data)
                if 'information' in data:
                    info_data = data['information']
                    info_data['user_id'] = data['user_id']
                    if not any(info_data.get(key) for key in ['sang', 'trua', 'toi', 'tai_kham']):
                        logger.warning(f"Bệnh nhân {data['user_id']} không có lịch nhắc nhở")
                    # Xác thực ngày tái khám
                    if info_data.get('tai_kham'):
                        try:
                            tai_kham_date = datetime.strptime(info_data['tai_kham'], "%Y-%m-%d").date()
                            current_date = datetime.now(timezone(timedelta(hours=7))).date()
                            if tai_kham_date <= current_date:
                                raise ValueError("Ngày tái khám phải sau ngày hiện tại!")
                        except ValueError:
                            raise ValueError("Định dạng ngày tái khám không hợp lệ (YYYY-MM-DD)!")
                    await insert_to_db('information', info_data)

                if data.get('telegram_token') and data['telegram_token'].lower() != 'nan':
                    try:
                        await get_bot(data['telegram_token'], is_private_bot=True, is_doctor_bot=False)
                    except Exception as e:
                        await bot.send_message(
                            chat_id=update.effective_chat.id,
                            text=f"⚠️ Đã thêm bệnh nhân nhưng lỗi khi khởi động bot riêng: {str(e)}",
                            parse_mode='Markdown'
                        )
                        context.user_data.pop('expecting', None)
                        return

                conn.commit()
                export_to_csv()

                info = data.get('information', {})
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 *Đã thêm bệnh nhân*:\n"
                        f"• *ID*: `{data['user_id']}`\n"
                        f"• *Tên*: {data['name']}\n"
                        f"• *Bác sĩ phụ trách*: `{data['doctor_id']}`\n"
                        f"• *Bot*: `{data['bot_name']}`\n"
                        f"• *Thông tin*:\n"
                        f"  - Tuổi: {info.get('tuoi', 'N/A')}\n"
                        f"  - Bệnh: {info.get('benh', 'N/A')}\n"
                        f"  - Thuốc: {info.get('thuoc', 'N/A')}\n"
                        f"  - Nhắc nhở uống thuốc:\n"
                        f"    + Sáng: {info.get('sang', 'N/A')}\n"
                        f"    + Trưa: {info.get('trua', 'N/A')}\n"
                        f"    + Tối: {info.get('toi', 'N/A')}\n"
                        f"  - Tái khám: {info.get('tai_kham', 'N/A')}\n"
                        f"📌 Bệnh nhân cần dùng `/cofirm {data['user_id']} <telegram_id> patient` để xác nhận vai trò."
                    ),
                    parse_mode='Markdown'
                )

                # Thông báo cho bác sĩ phụ trách
                if doctor[2]:  # doctor[2] là telegram_id
                    try:
                        doctor_bot = bot
                        if doctor[3] and doctor[3].lower() != 'nan':  # doctor[3] là telegram_token
                            doctor_bot = await get_bot(doctor[3], is_private_bot=True, is_doctor_bot=True)
                        await doctor_bot.send_message(
                            chat_id=doctor[2],
                            text=(
                                f"🧑 *Bệnh nhân mới được thêm*:\n"
                                f"• *ID*: `{data['user_id']}`\n"
                                f"• *Tên*: {data['name']}\n"
                                f"• *Bot*: `{data['bot_name']}`\n"
                                f"📌 Bệnh nhân cần xác nhận vai trò bằng `/cofirm`."
                            ),
                            parse_mode='Markdown'
                        )
                        logger.info(f"Đã gửi thông báo tới bác sĩ {data['doctor_id']} về bệnh nhân mới {data['user_id']}")
                    except Exception as e:
                        logger.error(f"Lỗi khi gửi thông báo tới bác sĩ {data['doctor_id']}: {e}")

                context.user_data.pop('expecting', None)
            finally:
                conn.close()
        except json.JSONDecodeError:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Dữ liệu không phải JSON hợp lệ. Vui lòng gửi đúng định dạng JSON!",
                parse_mode='Markdown'
            )
        except ValueError as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}",
                parse_mode='Markdown'
            )
        except Exception as e:
            logger.error(f"Lỗi khi xử lý /add_patient từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi không xác định: {str(e)}\nℹ️ Vui lòng kiểm tra lại dữ liệu hoặc liên hệ hỗ trợ.",
                parse_mode='Markdown'
            )
        return

    if expecting == 'doctor_log':
        try:
            lines = update.message.text.strip().split('\n')
            data = {'user_id': context.user_data.get('user_id')}
            for line in lines:
                if line.startswith('Địa chỉ:'):
                    data['address'] = line.replace('Địa chỉ:', '').strip()
                elif line.startswith('Số điện thoại:'):
                    phone = line.replace('Số điện thoại:', '').strip()
                    if not phone.isdigit() or len(phone) != 10:
                        raise ValueError("Số điện thoại phải là 10 chữ số!")
                    data['phone'] = phone
                elif line.startswith('Chuyên khoa:'):
                    data['specialty'] = line.replace('Chuyên khoa:', '').strip()

            if not all(key in data for key in ['address', 'phone', 'specialty']):
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text="❌ Thiếu `Địa chỉ`, `Số điện thoại` hoặc `Chuyên khoa`. Vui lòng gửi đúng định dạng:\n```\nĐịa chỉ: <địa chỉ>\nSố điện thoại: <10 chữ số>\nChuyên khoa: <chuyên khoa>\n```",
                    parse_mode='Markdown'
                )
                return

            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()
                cursor.execute("SELECT name, telegram_id FROM users WHERE user_id = ?", (data['user_id'],))
                user = cursor.fetchone()
                if not user:
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ Không tìm thấy bác sĩ với ID `{data['user_id']}`!",
                        parse_mode='Markdown'
                    )
                    return
                data['name'] = user[0]
                data['telegram_id'] = user[1]

                # Kiểm tra thông tin đã tồn tại
                cursor.execute("SELECT user_id FROM doctors_log WHERE user_id = ?", (data['user_id'],))
                if cursor.fetchone():
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=(
                            "⚠️ Thông tin bác sĩ đã tồn tại. Bạn có muốn ghi đè?\n"
                            "Gửi `Xác nhận` để tiếp tục hoặc `/cancel` để hủy."
                        ),
                        parse_mode='Markdown'
                    )
                    context.user_data['expecting'] = 'confirm_doctor_log'
                    context.user_data['doctor_log_data'] = data
                    return

                await insert_to_db('doctors_log', data)
                conn.commit()
                export_to_csv()

                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 *Đã cập nhật thông tin bác sĩ*:\n"
                        f"• *ID*: `{data['user_id']}`\n"
                        f"• *Tên*: {data['name']}\n"
                        f"• *Địa chỉ*: {data['address']}\n"
                        f"• *Số điện thoại*: {data['phone']}\n"
                        f"• *Chuyên khoa*: {data['specialty']}\n"
                        f"📌 Thông tin đã được lưu vào `doctors_log.csv`."
                    ),
                    parse_mode='Markdown'
                )
                context.user_data.pop('expecting', None)
                context.user_data.pop('user_id', None)
            finally:
                conn.close()
        except ValueError as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}",
                parse_mode='Markdown'
            )
        except Exception as e:
            logger.error(f"Lỗi khi xử lý thông tin bác sĩ từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi không xác định: {str(e)}\nℹ️ Vui lòng kiểm tra lại dữ liệu hoặc liên hệ hỗ trợ.",
                parse_mode='Markdown'
            )
        return

    if expecting == 'confirm_doctor_log':
        if update.message.text.strip().lower() == 'xác nhận':
            try:
                data = context.user_data.get('doctor_log_data')
                if not data:
                    raise ValueError("Không tìm thấy dữ liệu bác sĩ để ghi đè!")
                conn = sqlite3.connect(DB_PATH)
                try:
                    await insert_to_db('doctors_log', data)
                    conn.commit()
                    export_to_csv()

                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=(
                            f"🎉 *Đã cập nhật thông tin bác sĩ*:\n"
                            f"• *ID*: `{data['user_id']}`\n"
                            f"• *Tên*: {data['name']}\n"
                            f"• *Địa chỉ*: {data['address']}\n"
                            f"• *Số điện thoại*: {data['phone']}\n"
                            f"• *Chuyên khoa*: {data['specialty']}\n"
                            f"📌 Thông tin đã được lưu vào `doctors_log.csv`."
                        ),
                        parse_mode='Markdown'
                    )
                    context.user_data.pop('expecting', None)
                    context.user_data.pop('user_id', None)
                    context.user_data.pop('doctor_log_data', None)
                finally:
                    conn.close()
            except ValueError as e:
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=f"❌ Lỗi: {str(e)}",
                    parse_mode='Markdown'
                )
            except Exception as e:
                logger.error(f"Lỗi khi ghi đè thông tin bác sĩ từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=f"❌ Lỗi không xác định: {str(e)}\nℹ️ Vui lòng kiểm tra lại hoặc liên hệ hỗ trợ.",
                    parse_mode='Markdown'
                )
        else:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Đã hủy cập nhật thông tin bác sĩ.",
                parse_mode='Markdown'
            )
            context.user_data.pop('expecting', None)
            context.user_data.pop('user_id', None)
            context.user_data.pop('doctor_log_data', None)
        return

    if expecting == 'prescription':
        try:
            data = json.loads(update.message.text.strip())
            if not isinstance(data, dict):
                raise ValueError("Dữ liệu phải là JSON object")

            required_fields = ['benh', 'thuoc']
            if not all(field in data for field in required_fields):
                raise ValueError("Thiếu `benh` hoặc `thuoc`")

            patient_id = context.user_data.get('patient_id')
            if not patient_id:
                raise ValueError("Không tìm thấy ID bệnh nhân!")
            data['user_id'] = patient_id
            conn = sqlite3.connect(DB_PATH)
            try:
                cursor = conn.cursor()

                # Kiểm tra bệnh nhân tồn tại và bác sĩ phụ trách
                cursor.execute("SELECT name, telegram_id, doctor_id FROM users WHERE user_id = ?", (patient_id,))
                patient = cursor.fetchone()
                if not patient:
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ Không tìm thấy bệnh nhân với ID `{patient_id}`!",
                        parse_mode='Markdown'
                    )
                    return

                if user['user_id'] != patient[2]:
                    await bot.send_message(
                        chat_id=update.effective_chat.id,
                        text=f"❌ Bạn không phải bác sĩ phụ trách bệnh nhân này!",
                        parse_mode='Markdown'
                    )
                    return

                # Xác thực ngày tái khám
                if data.get('tai_kham'):
                    try:
                        tai_kham_date = datetime.strptime(data['tai_kham'], "%Y-%m-%d").date()
                        current_date = datetime.now(timezone(timedelta(hours=7))).date()
                        if tai_kham_date <= current_date:
                            raise ValueError("Ngày tái khám phải sau ngày hiện tại!")
                    except ValueError:
                        raise ValueError("Định dạng ngày tái khám không hợp lệ (YYYY-MM-DD)!")

                await insert_to_db('information', data)
                conn.commit()
                export_to_csv()

                await bot.send_message(
                    chat_id=update.effective_chat.id,
                    text=(
                        f"🎉 *Đã cập nhật đơn thuốc cho bệnh nhân {patient[0]}* (ID: `{patient_id}`):\n"
                        f"• *Bệnh*: {data['benh']}\n"
                        f"• *Thuốc*: {data['thuoc']}\n"
                        f"• *Nhắc nhở uống thuốc*:\n"
                        f"  - Sáng: {data.get('sang', 'N/A')}\n"
                        f"  - Trưa: {data.get('trua', 'N/A')}\n"
                        f"  - Tối: {data.get('toi', 'N/A')}\n"
                        f"• *Tái khám*: {data.get('tai_kham', 'N/A')}"
                    ),
                    parse_mode='Markdown'
                )

                if patient[1]:
                    try:
                        target_bot = bot
                        patient_user = get_user_by_id(patient_id)
                        if patient_user and patient_user.get('telegram_token') and patient_user['telegram_token'].lower() != 'nan':
                            target_bot = await get_bot(patient_user['telegram_token'], is_private_bot=True, is_doctor_bot=False)
                        await target_bot.send_message(
                            chat_id=patient[1],
                            text=(
                                f"📝 *Đơn thuốc mới*:\n"
                                f"• *Bệnh*: {data['benh']}\n"
                                f"• *Thuốc*: {data['thuoc']}\n"
                                f"• *Nhắc nhở uống thuốc*:\n"
                                f"  - Sáng: {data.get('sang', 'N/A')}\n"
                                f"  - Trưa: {data.get('trua', 'N/A')}\n"
                                f"  - Tối: {data.get('toi', 'N/A')}\n"
                                f"• *Tái khám*: {data.get('tai_kham', 'N/A')}\n"
                                f"📌 Vui lòng tuân thủ hướng dẫn của bác sĩ!"
                            ),
                            parse_mode='Markdown'
                        )
                        logger.info(f"Đã gửi thông báo đơn thuốc mới tới bệnh nhân {patient[0]} (ID: {patient_id})")
                    except Exception as e:
                        logger.error(f"Lỗi khi gửi thông báo đơn thuốc tới bệnh nhân {patient[0]} (ID: {patient_id}): {e}")

                context.user_data.pop('expecting', None)
                context.user_data.pop('patient_id', None)
            finally:
                conn.close()
        except json.JSONDecodeError:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Dữ liệu không phải JSON hợp lệ. Vui lòng gửi đúng định dạng JSON!",
                parse_mode='Markdown'
            )
        except ValueError as e:
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi: {str(e)}",
                parse_mode='Markdown'
            )
        except Exception as e:
            logger.error(f"Lỗi khi xử lý /update_prescription từ Telegram ID {telegram_id}: {str(e)}", exc_info=True)
            await bot.send_message(
                chat_id=update.effective_chat.id,
                text=f"❌ Lỗi không xác định: {str(e)}\nℹ️ Vui lòng kiểm tra lại dữ liệu hoặc liên hệ hỗ trợ.",
                parse_mode='Markdown'
            )
        return
# --- Xử lý lỗi ---
async def error_handler(update: Update, context):
    """Xử lý lỗi xảy ra trong bot Telegram."""
    logger.error(f"Lỗi khi xử lý cập nhật {update}: {context.error}", exc_info=True)
    if update:
        try:
            await context.bot.send_message(
                chat_id=update.effective_chat.id,
                text="❌ Đã xảy ra lỗi. Vui lòng thử lại hoặc liên hệ hỗ trợ!",
                parse_mode='Markdown'
            )
        except Exception as e:
            logger.error(f"Lỗi khi gửi thông báo lỗi tới người dùng: {e}")

# --- FastAPI ---
app = FastAPI(title="Telegram Bot API")
security = HTTPBearer()

class DoctorInfo(BaseModel):
    user_id: int
    name: str
    address: Optional[str] = None
    phone: Optional[str] = None
    specialty: Optional[str] = None

async def verify_token(credentials: HTTPAuthorizationCredentials = Depends(security)):
    """Xác thực token API."""
    if credentials.credentials != DOCTOR_API_TOKEN:
        raise HTTPException(status_code=401, detail="Token không hợp lệ")
    return credentials.credentials

@app.get("/doctor/{doctor_id}", response_model=DoctorInfo)
async def get_doctor_info_api(doctor_id: int, token: str = Depends(verify_token)):
    """API lấy thông tin bác sĩ từ doctors_log."""
    try:
        doctor_log = get_doctor_log(doctor_id)
        if not doctor_log:
            raise HTTPException(status_code=404, detail=f"Không tìm thấy bác sĩ với ID {doctor_id}")
        return DoctorInfo(**doctor_log)
    except Exception as e:
        logger.error(f"Lỗi khi lấy thông tin bác sĩ {doctor_id} qua API: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/doctor", response_model=DoctorInfo)
async def add_doctor_info_api(doctor: DoctorInfo, token: str = Depends(verify_token)):
    """API thêm hoặc cập nhật thông tin bác sĩ vào doctors_log."""
    try:
        conn = sqlite3.connect(DB_PATH)
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM users WHERE user_id = ? AND role = 'doctor'", (doctor.user_id,))
            user = cursor.fetchone()
            if not user:
                raise HTTPException(status_code=404, detail=f"Không tìm thấy bác sĩ với ID {doctor.user_id}")

            data = {
                'user_id': doctor.user_id,
                'name': user[0],
                'telegram_id': None,  # API không cập nhật telegram_id
                'address': doctor.address,
                'phone': doctor.phone,
                'specialty': doctor.specialty
            }
            await insert_to_db('doctors_log', data)
            conn.commit()
            export_to_csv()
            return DoctorInfo(**data)
        finally:
            conn.close()
    except Exception as e:
        logger.error(f"Lỗi khi thêm bác sĩ qua API: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# --- Dọn dẹp khi thoát ---
async def cleanup_bots():
    """Dừng tất cả bot khi ứng dụng kết thúc."""
    global bots
    for token, bot_data in list(bots.items()):
        try:
            await bot_data['application'].updater.stop()
            await bot_data['application'].stop()
            await bot_data['application'].shutdown()
            logger.info(f"Đã dừng bot với token {token}")
        except Exception as e:
            logger.error(f"Lỗi khi dừng bot {token}: {e}")
    bots.clear()

atexit.register(lambda: asyncio.run(cleanup_bots()))

async def main():
    try:
        # Khởi tạo cơ sở dữ liệu
        init_db()

        # Khởi chạy bot chính
        main_bot = await get_bot(DEFAULT_TELEGRAM_TOKEN, is_private_bot=False)
        logger.info("Bot chính đã khởi động")

        # Lên lịch nhắc nhở
        schedule_reminders()

        # Khởi chạy các bot riêng đã lưu
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT telegram_token, role FROM users WHERE telegram_token IS NOT NULL AND telegram_token != ?",
                      (DEFAULT_TELEGRAM_TOKEN,))
        existing_bots = cursor.fetchall()
        conn.close()

        for token, role in existing_bots:
            if token and token.lower() != 'nan':
                try:
                    await get_bot(token, is_private_bot=True, is_doctor_bot=role == 'doctor')
                    logger.info(f"Đã khởi động bot riêng với token {token}")
                except Exception as e:
                    logger.error(f"Lỗi khi khởi động bot riêng với token {token}: {e}")

        # FastAPI sẽ được chạy bởi uvicorn
        logger.info("Ứng dụng đã sẵn sàng. Chạy FastAPI với uvicorn trên cổng 8000.")
    except Exception as e:
        logger.error(f"Lỗi khi khởi động ứng dụng: {e}")
        await cleanup_bots()
        raise

if __name__ == "__main__":
    import uvicorn
    loop = asyncio.get_event_loop()
    loop.create_task(main())
    uvicorn.run(app, host="0.0.0.0", port=8000)

Mounted at /content/drive


INFO:     Started server process [678]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^